In [1]:
# 所有的通用变量都放在这里,方便管理

# x_list = ['6908','6909','6910','6911','6912','6913','9783','12066']
# x_list = ['6914', '6916', '6917', '6918', '9775', '9776', '9777']
x_list = ['1','2','3']
y_list = ['txt', 'price']
path = 'Lv2期结论'

# filter_layer_cases = [[1.0], [2.0], [3.0], [4.0], [5.0], [6.0]]
filter_layer_cases = [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]


# filter_suffix_list = ['filter_1.0_2.0_3.0', 'filter_4.0_5.0_6.0']
# filter_suffix_list = ['filter_1.0','filter_3.0','filter_3.0','filter_4.0''filter_5.0' 'filter_6.0']
filter_suffix_list = ['filter_1.0_2.0', 'filter_3.0_4.0', 'filter_5.0_6.0']

z = '男鞋_新分类_from_0501'




# step1 - 对于图片主体的box数量统计

In [ ]:
import os
import pandas as pd
import re



def process_excel(input_file, output_file):
    # 读取Excel文件
    df = pd.read_excel(input_file)
    df = df.drop_duplicates()

    # 定义一个函数来移除后缀
    def remove_suffix(name):
        return re.sub(r'_\d+$', '', name)

    # 应用函数到'Image Name'列
    df['Image Name'] = df['Image Name'].apply(remove_suffix)

    # 计算每个Image Name的出现次数
    name_counts = df['Image Name'].value_counts()

    # 创建一个新的'box_no'列，并填充对应的计数
    df['box_no'] = df['Image Name'].map(name_counts)

    # 保存修改后的DataFrame到新的Excel文件
    df.to_excel(output_file, index=False)

    print(f"处理完成，结果已保存到 {output_file}")

def process_all_folders(base_path):
    for root, dirs, files in os.walk(base_path):
        if 'grounding_output' in dirs:
            grounding_output_path = os.path.join(root, 'grounding_output')
            input_file = os.path.join(grounding_output_path, 'grounding_results.xlsx')
            if os.path.exists(input_file):
                output_file = os.path.join(grounding_output_path, 'grounding_results_processed.xlsx')
                process_excel(input_file, output_file)

# 设置基础路径
base_path = f'D://code//data//{path}//{z}'

# 处理所有文件夹
process_all_folders(base_path)

print("所有文件夹处理完成")

import datetime
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")




# step1.1 - 在图片信息统计的基础上拼接品牌分成信息

In [ ]:
# 添加品牌分层信息


import os
import pandas as pd
import re


csv_file_path = f'D://code//data//{path}//{z}//{z}.csv'
brand_path = f'D://code//data//{path}//{z}//女鞋品牌分层.xlsx'

# 读取CSV文件=
df = pd.read_csv(csv_file_path)
df_brand = pd.read_excel(brand_path)

# 合并df和df_brand数据
df = pd.merge(df, df_brand, on='main_brand_code', how='left')

# # 定义筛选条件
filter_layers = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]


# 筛选数据
filtered_df = df[df['最终分层'].isin(filter_layers)]

def extract_matching_part(img_url):
    if pd.isna(img_url):
        return None
    img_url = img_url.split('?')[0]
    img_url = os.path.splitext(img_url)[0]
    parts = img_url.split('/')
    if len(parts) >= 2:
        return f"{parts[-2]}_{parts[-1]}"
    return None

filtered_df['matching_part'] = filtered_df['img_url'].apply(extract_matching_part)
filtered_df = filtered_df[['matching_part', 'main_brand_code', '最终分层']]

def process_excel(input_file, filtered_df):
    # 读取Excel文件
    df = pd.read_excel(input_file)

    # 先对DataFrame进行去重
    df = df.drop_duplicates()

    # 定义一个函数来移除后缀
    def remove_suffix(name):
        return re.sub(r'_\d+$', '', name)

    # 应用函数到'Image Name'列
    df['Image Name'] = df['Image Name'].apply(remove_suffix)

    # 计算每个Image Name的出现次数
    name_counts = df['Image Name'].value_counts()

    # 创建一个新的'box_no'列，并填充对应的计数
    df['box_no'] = df['Image Name'].map(name_counts)

    # 将filtered_df中的数据添加到df中
    df = pd.merge(df, filtered_df, left_on='Image Name', right_on='matching_part', how='left')

    # 删除matching_part列，因为它与Image Name重复
    df = df.drop(columns=['matching_part'])

    return df

def process_all_folders(base_path, filtered_df):
    for root, dirs, files in os.walk(base_path):
        if 'grounding_output' in dirs:
            grounding_output_path = os.path.join(root, 'grounding_output')
            input_file = os.path.join(grounding_output_path, 'grounding_results.xlsx')
            if os.path.exists(input_file):
                processed_df = process_excel(input_file, filtered_df)
                output_file = os.path.join(grounding_output_path, 'grounding_results_processed.xlsx')
                processed_df = processed_df.drop_duplicates()
                processed_df.to_excel(output_file, index=False)
                print(f"处理完成，结果已保存到 {output_file}")

# 设置基础路径
base_path = f'D://code//data//{path}//{z}'

# 处理所有文件夹
process_all_folders(base_path, filtered_df)

print("所有文件夹处理完成")

import datetime
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")




# step2 - 文本框识别, 合并相邻的文本框, 生成txt_info

In [2]:
# 修改后的代码, 先从图片中识别出文本, 然后分两步
# ① 对文本框进行阈值下的合并; 同时也保留原文本框
# ② 对文本进行高度和关键词的分类



import os
import glob
from tqdm import tqdm
import pandas as pd
from paddleocr import PaddleOCR
from PIL import Image
import math
import re



# 设置输入和输出路径
input_folder_path = f'D://code//data//{path}//{z}'
output_file_path = f'D://code//data//{path}//{z}//txt_info.xlsx'

# 加载 OCR 模型
ocr = PaddleOCR(use_angle_cls=True, lang="ch", show_log=False)

def calculate_shortest_distance(point_a, points_bcd):
    shortest_distance = float('inf')
    for point_bcd in points_bcd:
        distance = ((point_bcd[0] - point_a[0]) ** 2 + (point_bcd[1] - point_a[1]) ** 2) ** 0.5
        if distance < shortest_distance:
            shortest_distance = distance
    return shortest_distance

def merge_text_boxes(img_path, style):
    result = ocr.ocr(img_path, cls=True)
    img = Image.open(img_path)
    img_width, img_height = img.size

    if not result or not result[0]:
        # print(f"No text detected in the image: {img_path}")
        return None, None

    rectangles_with_text = result[0]

    original_text_box_info = []
    for rectangle in rectangles_with_text:
        points = rectangle[0]
        original_text_box_info.append({
            'File Name': os.path.basename(img_path),
            'Style': style,
            'x1': points[0][0],
            'y1': points[0][1],
            'x2': points[2][0],
            'y2': points[2][1],
            'text': rectangle[1][0]
        })

    merged_text_boxes = []

    for index, row in pd.DataFrame(original_text_box_info).iterrows():
        if not merged_text_boxes:
            merged_text_boxes.append(row.to_dict())
        else:
            last_merged_box = merged_text_boxes[-1]

            if calculate_shortest_distance((row['x1'], row['y1']), [(last_merged_box['x1'], last_merged_box['y1']), (last_merged_box['x2'], last_merged_box['y1']), (last_merged_box['x2'], last_merged_box['y2']), (last_merged_box['x1'], last_merged_box['y2'])]) < 100:  # 设定文本框合并的阈值
                last_merged_box['text'] += ' ' + row['text']
                last_merged_box['x1'] = min(last_merged_box['x1'], row['x1'])
                last_merged_box['y1'] = min(last_merged_box['y1'], row['y1'])
                last_merged_box['x2'] = max(last_merged_box['x2'], row['x2'])
                last_merged_box['y2'] = max(last_merged_box['y2'], row['y2'])
            else:
                merged_text_boxes.append(row.to_dict())

    original_text_box_df = pd.DataFrame(original_text_box_info)
    merged_text_box_df = pd.DataFrame(merged_text_boxes)

    for i, box in original_text_box_df.iterrows():
        if box['y1'] < img_height / 2 and box['y2'] < img_height / 2:
            region = '上半'
        elif box['y1'] >= img_height / 2 and box['y2'] >= img_height / 2:
            region = '下半'
        elif box['x1'] < img_width / 2 and box['x2'] < img_width / 2:
            region = '左半'
        else:
            region = '右半'
        original_text_box_df.at[i, 'Region'] = region

        box_area = (box['x2'] - box['x1']) * (box['y2'] - box['y1'])
        box_per = box_area / (img_width * img_height)
        original_text_box_df.at[i, 'txt_Area'] = box_area
        original_text_box_df.at[i, 'txt_Per'] = box_per

    for i, box in merged_text_box_df.iterrows():
        if box['y1'] < img_height / 2 and box['y2'] < img_height / 2:
            region = '上半'
        elif box['y1'] >= img_height / 2 and box['y2'] >= img_height / 2:
            region = '下半'
        elif box['x1'] < img_width / 2 and box['x2'] < img_width / 2:
            region = '左半'
        else:
            region = '右半'
        merged_text_box_df.at[i, 'Region'] = region

        merge_area = (box['x2'] - box['x1']) * (box['y2'] - box['y1'])
        merge_per = merge_area / (img_width * img_height)
        merged_text_box_df.at[i, 'Area'] = merge_area
        merged_text_box_df.at[i, 'Per'] = merge_per

    return merged_text_box_df, original_text_box_df

keyword_groups = {
    '通用': ['以旧换新', '只换不修', '包邮', '无理由退', '先用后付', '京东白条', '期免息', '送货上门', '保修'],
    '价保': ['价保', '保价'],
    '纯价格': ['¥', '夫', '￥', r'\b价\b', '到手价', '活动价'],
    '直降': ['立减', '直降', '降', '立省', r'^(?!.*升降).*$', r'^(?!.*降温).*$', r'^(?!.*降噪).*$', r'^(?!.*降低).*$'],
    '折扣': ['折', r'^(?!.*折叠).*$', r'^(?!.*翻折).*$'],
    '满减': [r'.*满.*减.*', r'.*满.*-.*', r'.*满.*免.*'],
    '用券': ['用券', '领券', '券'],
    '返券': ['返券', '京豆', '返现', r'.*返.*E卡.*', r'.*返.*红包.*'],
    '限时': ['.*小时$', '.*天$', '时间', 'time', 'TIME', '限时', r'.*月.*日.*', r'.*日.*点.*', r'.*:.*', r'.*:.*', r'.*：.*', r'\b\d{1,2}\.\d{1,2}-\d{1,2}\b'],
    'xx元任选': [r'.*元.*件.*'],
    '赠品': [r'.*满.*赠.*', r'.*满.*送.*', '送', '抽', '奖励', '赠', r'^(?!.*送货).*$', r'^(?!.*送礼).*$', r'^(?!.*送装).*$', r'^(?!.*配送).*$', r'^(?!.*送达).*$'],
    '节日名称': ['节', '出游季', '购物季', '毕业季', '开学季', '黑五', '周年庆', '儿童节', '父亲节', '端午节', '七夕', '中秋节', '国庆', '万圣节', '感恩节', '元旦', '圣诞', '情人节', '春节', '元宵节', '38节', '3.8节', '清明节', '母亲节', '618', '购物季', '开学季', '11.11', '黑五', '12.12', '女神节', '出游季', '放价季', '吃货节', '家装节'],
    '是否限购': ['限购', '限量']
}

def keyword_analysis(text):
    results = {}
    for key, words in keyword_groups.items():
        results[key] = any(re.search(word, text) for word in words)
    return results

def height_analysis(x1, y1, x2, y2):
    height = abs(y2 - y1)
    return height

def process_images(folder_path, subfolder_name):
    image_files = []
    for root, dirs, files in os.walk(folder_path):
        if 'grounding_output' in root and ('price' in root or 'txt' in root):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                    image_files.append(os.path.join(root, file))
    
    combined_results = []
    for img_path in tqdm(image_files, desc=f'Processing images in {subfolder_name}'):
        style = 'price' if 'price' in img_path else 'txt'
        merged_df, original_df = merge_text_boxes(img_path, style)
        if merged_df is not None and original_df is not None:
            merged_df['Subfolder'] = subfolder_name
            original_df['Subfolder'] = subfolder_name
            combined_results.append({
                'original': original_df,
                'merged': merged_df
            })
    
    return combined_results

# 主程序
all_results = []
for folder in os.listdir(input_folder_path):
    if folder.isdigit():
        folder_path = os.path.join(input_folder_path, folder)
        if os.path.isdir(folder_path):
            print(f"正在处理文件夹: {folder_path}")
            results = process_images(folder_path, folder)
            all_results.extend(results)

final_combined_data = []
for result in all_results:
    result['original']['Type'] = 'Original'
    result['merged']['Type'] = 'Merged'
    combined = pd.concat([result['original'], result['merged']], ignore_index=True)
    final_combined_data.append(combined)

final_combined_df = pd.concat(final_combined_data, ignore_index=True)
final_combined_df.sort_values(by=['Subfolder', 'File Name', 'Type'], inplace=True)

for index, row in tqdm(final_combined_df.iterrows(), total=final_combined_df.shape[0], desc="Analyzing text"):
    keyword_results = keyword_analysis(row['text'])
    for key, value in keyword_results.items():
        final_combined_df.at[index, key] = value
    
    height = height_analysis(row['x1'], row['y1'], row['x2'], row['y2'])
    final_combined_df.at[index, 'Height'] = height
    final_combined_df.at[index, 'Height_Category'] = (
        'Height_<18' if height < 18 else
        'Height_18-29' if 18 <= height < 29 else
        'Height_29-38' if 29 <= height < 38 else
        'Height_>38'
    )

final_combined_df.to_excel(output_file_path, index=False)

print('处理完成')

import datetime
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")




正在处理文件夹: D://code//data//Lv2期结论//男鞋_新分类_from_0501\1


Processing images in 1:   2%|▏         | 21/1243 [00:05<05:09,  3.95it/s]

: 

: 

# 2.1 - 在现有txt_info的基础上, 拼接品牌分类, 生成txt_info-1

In [ ]:
# 在现有txt_info的基础上, 拼接品牌分类
# 在现有txt_info的基础上, 拼接品牌分类
# 在现有txt_info的基础上, 拼接品牌分类


import os
import glob
from tqdm import tqdm
import pandas as pd
from paddleocr import PaddleOCR
from PIL import Image
import math
import re



in_file_path = f'D://code//data//{path}//{z}//txt_info.xlsx'
output_file_path = f'D://code//data//{path}//{z}//txt_info-1.xlsx'

df = pd.read_excel(in_file_path)

# 删除df的File Name列中的.jpg扩展名
df['File Name'] = df['File Name'].str.replace('.jpg', '')

filtered_df = filtered_df.drop_duplicates()

# 删除filtered_df的matching_part列中的.jpg扩展名
filtered_df['matching_part'] = filtered_df['matching_part'].str.replace('.jpg', '')

df1 = pd.merge(df, filtered_df, left_on='File Name', right_on='matching_part', how='left')

df1.to_excel(output_file_path)

import datetime
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")




# step3 - 将分散在各个grounding_output文件夹中的grounding_results_processed合并为img_info

In [ ]:
import os
import pandas as pd
import openpyxl

def merge_excel_files(base_path):
    # 用于存储所有数据框的列表
    all_dataframes = []

    # 遍历基础路径下的所有文件夹
    for root, dirs, files in os.walk(base_path):
        if 'grounding_output' in dirs:
            grounding_output_path = os.path.join(root, 'grounding_output')
            excel_file = os.path.join(grounding_output_path, 'grounding_results_processed.xlsx')
            
            if os.path.exists(excel_file):
                # 读取Excel文件
                df = pd.read_excel(excel_file)
                
                # 添加新列，值为当前子文件夹的名称
                subfolder_name = os.path.basename(os.path.dirname(grounding_output_path))
                df['Subfolder'] = subfolder_name
                
                # 将数据框添加到列表中
                all_dataframes.append(df)

    # 合并所有数据框
    if all_dataframes:
        merged_df = pd.concat(all_dataframes, ignore_index=True)
        
        # 保存合并后的数据框到新的Excel文件
        output_file = os.path.join(base_path, 'img_info.xlsx')
        merged_df.to_excel(output_file, index=False)
        print(f"合并完成，结果保存在: {output_file}")
    else:
        print("没有找到符合条件的Excel文件")

# 使用示例
base_path = f"D://code//data//{path}//{z}"
merge_excel_files(base_path)

import datetime
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")




# step3.1 - 在img_info后面拼接品牌信息, 生成img_info-1

In [ ]:
# 在img_info后面拼接品牌分类信息


# z = '女士春夏上装_from_0501'

in_file_path = f'D://code//data//{path}//{z}//img_info.xlsx'
output_file_path = f'D://code//data//{path}//{z}//img_info-1.xlsx'

df = pd.read_excel(in_file_path)

# 删除df的File Name列中的.jpg扩展名
df['Image Name'] = df['Image Name'].str.replace('.jpg', '')

filtered_df = filtered_df.drop_duplicates()

# 删除filtered_df的matching_part列中的.jpg扩展名
filtered_df['matching_part'] = filtered_df['matching_part'].str.replace('.jpg', '')

df1 = pd.merge(df, filtered_df, left_on='Image Name', right_on='matching_part', how='left')
df1 = df1.drop_duplicates()

df1.to_excel(output_file_path)
print(output_file_path)

import datetime
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")
print(f"完成时间: {formatted_time}")




# step4 - 将布局和ctr参数进行合并

In [ ]:
import pandas as pd
import cv2
import numpy as np
from PIL import Image
import os



def merge_excel_files(txt_box_info_file, img_box_info_file, output_file):
    # 读取 txt_box_info 文件
    txt_df = pd.read_excel(txt_box_info_file)
    txt_df = txt_df.rename(columns={'File Name': 'Image Name', 'x1': 'txt_x1', 'y1': 'txt_y1', 'x2': 'txt_x2', 'y2': 'txt_y2'})

    # 读取 img_box_info 文件
    img_df = pd.read_excel(img_box_info_file)

    # 重命名列
    img_df = img_df.rename(columns={'x1': 'img_x1', 'y1': 'img_y1', 'x3': 'img_x2', 'y3': 'img_y2', 'Subfolder': 'Style'})
    img_df = img_df.loc[:, ['Image Name', 'Style', 'img_x1', 'img_y1', 'img_x2', 'img_y2', 'box_no']]
    img_df = img_df.drop_duplicates()

    # 合并两个DataFrame，以'Image Name'为键进行合并
    merged_df = pd.merge(txt_df, img_df, on='Image Name', how='outer')

    # 将 img_box_info 中缺少的数据设置为空
    merged_df = merged_df.fillna("")

    # 使用正则表达式删除.jpg 或.png 后缀
    merged_df['Image Name'] = merged_df['Image Name'].str.replace(r'\.(?:jpg|png)$', '', regex=True)

    # 将合并后的DataFrame写入新的Excel文件
    with pd.ExcelWriter(output_file) as writer:
        merged_df.to_excel(writer, index=False)


if __name__ == '__main__':
    txt_box_info_file = f"D://code//data//{path}//{z}//txt_info-1.xlsx"
    img_box_info_file = f"D://code//data//{path}//{z}//img_info.xlsx"
    output_file = f"D://code//data//{path}//{z}//merged_info-1.xlsx"

    merge_excel_files(txt_box_info_file, img_box_info_file, output_file)

    print("Excel files merged successfully!")



import os
import pandas as pd
import re

# 定义路径
data_1 = f"D://code//data//{path}//{z}//merged_info-1.xlsx"
data_2 = f"D://code//data//{path}//{z}//{z}.csv"
# data_3 = 'D://code//data//howtodo_from_0401//服饰鞋靴箱包//品类聚类-服饰鞋靴箱包.csv'
output_path = f"D://code//data//{path}//{z}//merged_info_ctr-1.xlsx"

# 读取df1
df1 = pd.read_excel(data_1)

# 读取df2
df2 = pd.read_csv(data_2)

aggregated_data = df2.groupby('img_url').agg({
    'cid2': 'first',  # 使用 'first' 函数来选择分组中的第一个值
    'cid3': 'first',
    'uv': 'sum',
    'click_uv': 'sum',
    'gmv_cj':'sum',
    'sale_qtty_cj':'sum',
    'sku':'first'
#     'folder_path': 'first'  # 同样使用 'first' 函数选择第一个值
}).reset_index()  # 重置索引

df2 = aggregated_data

# 计算ctr字段
df2['ctr'] = df2['click_uv'] / df2['uv']


def extract_filename(x):
    # 分割路径，取倒数第二部分和最后一部分（文件名部分）
    parts = x.split('/')
    return f"{parts[-2]}_{os.path.splitext(parts[-1])[0]}"  # 保留原文件扩展名
    # return f"{os.path.splitext(parts[-1])[0]}.jpg"  # 保留原文件扩展名

# 应用函数
df2['only_2'] = df2['img_url'].apply(extract_filename)

# 初始化结果列表
results = []

# 遍历df1的每一行
for index, row1 in df1.iterrows():
    # 查找df2中匹配的行
    matching_rows_df2 = df2[df2['only_2'] == row1['Image Name']]
    
    # 如果没有找到匹配的行，则只添加df1的当前行
    if matching_rows_df2.empty:
        results.append(row1.to_dict())
    else:
        # 对于找到的每个匹配行，先添加df1的当前行，然后添加匹配的df2行
        results.append(row1.to_dict())
        for _, row2 in matching_rows_df2.iterrows():
            # 可能需要添加额外的逻辑来处理多个匹配的情况
            # 这里假设每个df1的行在df2中最多只有一个匹配
            merged_row = {**row1.to_dict(), **row2.to_dict()}
            results.append(merged_row)

# 将结果列表转换为DataFrame
result_df = pd.DataFrame(results)

result_df_drop = result_df.dropna(subset=['uv'])

# 保存到指定路径
result_df_drop.to_excel(output_path, index=False)

print(f"Merged file saved to {output_path}")

import datetime

current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

print(formatted_time)
print(formatted_time)
print(formatted_time)




# step5 - 将图片按照比例进行分类
### x<0.77 / 0.77<x<1.3 / x>1.3

In [ ]:
# # 这是用来对图片尺寸进行分类的,但是感觉在实际项目中并没有什么用,暂时注销了

# import pandas as pd
# import numpy as np
# import os
# import shutil
# from PIL import Image


# # 读取Excel文件
# df = pd.read_excel(f'D://code//data//{path}//{z}//img_info-1.xlsx')


# # 定义一个函数来计算新的矩形框坐标
# def calculate_new_coordinates(group):

#     min_x = group['x1'].min()
#     min_y = group['y1'].min()
#     max_x = group['x3'].max()
#     max_y = group['y3'].max()
    
#     return pd.Series({
#         'merge_x1': min_x,
#         'merge_y1': min_y,
#         'merge_x2': max_x,
#         'merge_y2': max_y
#     })


# # 按File Name分组并计算新坐标
# df = df.groupby('Image Name').apply(calculate_new_coordinates).reset_index()

# # 计算矩形框的横纵比
# df['aspect_ratio'] = (df['merge_x2'] - df['merge_x1']) / (df['merge_y2'] - df['merge_y1'])


# # 定义一个函数来分类图片并复制到相应文件夹
# def classify_and_copy_image(row):
#     # image_path = os.path.join(path_to_your_images, f"{row['File Name']}")
#     if row['aspect_ratio'] < 0.77:
#         # shutil.copy(image_path, os.path.join(path_to_your_images, '小于0.77', f"{row['File Name']}.jpg"))
#         return '小于0.77'
#     elif 0.77 <= row['aspect_ratio'] <= 1.3:
#         # shutil.copy(image_path, os.path.join(path_to_your_images, '0.77到1.3', f"{row['File Name']}.jpg"))
#         return '0.77到1.3'
#     else:
#         # shutil.copy(image_path, os.path.join(path_to_your_images, '大于1.3', f"{row['File Name']}.jpg"))
#         return '大于1.3'

# # 应用分类函数并添加结果列
# df['classification'] = df.apply(classify_and_copy_image, axis=1)

# # 保存结果到Excel
# output_file = os.path.join(f'D://code//data//{path}//{z}//0.77-1.3-1.xlsx')
# df.to_excel(output_file, index=False)

# print(f"处理完成，结果已保存到 {output_file}")

# import datetime

# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

# print(formatted_time)
# print(formatted_time)
# print(formatted_time)




# step6 - 针对商品主体的坐标和分类统计

In [ ]:
# 没有考虑brand
# 这里的热力图不是基于25宫格分类,而是基于商品数量
# 还是保留了5x5网格分类,生成了线框图和分类图片
# 将结果统计为了一个新的excel表,包含'x', 'y', 'box_no', 'x1', 'y1', 'x2', 'y2'



import pandas as pd
import os
import shutil
from tqdm import tqdm
import concurrent.futures
import itertools
import datetime
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt



# 用于收集每个 x 和 y 分类的结果
all_results = []

# 1. 归一化坐标
def normalize_coordinates(row):
    width, height = 800, 800
    row['left_norm'] = max(0, min(row['img_x1'] / width, 1))
    row['top_norm'] = max(0, min(row['img_y1'] / height, 1))
    row['right_norm'] = max(0, min(row['img_x2'] / width, 1))
    row['bottom_norm'] = max(0, min(row['img_y2'] / height, 1))
    return row

# 2. 判断重叠和分类
def check_overlap(rect, grid_cell):
    return not (rect[2] < grid_cell[0] or rect[0] > grid_cell[2] or
                rect[3] < grid_cell[1] or rect[1] > grid_cell[3])

def classify_image(group):
    overlaps = [0] * 25
    for _, row in group.iterrows():
        rect = (row['left_norm'], row['top_norm'], row['right_norm'], row['bottom_norm'])
        for i in range(5):
            for j in range(5):
                grid_cell = (i / 5, j / 5, (i + 1) / 5, (j + 1) / 5)
                if check_overlap(rect, grid_cell):
                    overlaps[i * 5 + j] = 1
    return ''.join(map(str, overlaps))

# 3. 绘制线框图
def draw_rectangles(group, save_path):
    fig, ax = plt.subplots(figsize=(6.16, 6.16), dpi=100)

    # 绘制矩形框
    for _, row in group.iterrows():
        rect = Rectangle((row['left_norm'], 1 - row['bottom_norm']),
                         row['right_norm'] - row['left_norm'],
                         row['bottom_norm'] - row['top_norm'],
                         fill=False, edgecolor='r')
        ax.add_patch(rect)

    # 绘制5x5网格
    for i in range(5):
        for j in range(5):
            grid_rect = Rectangle((j / 5, 1 - (i + 1) / 5), 1 / 5, 1 / 5, fill=False, edgecolor='b')
            ax.add_patch(grid_rect)

    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')

    # 保存线框图
    fig.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

# 4. 处理单个图像并分类
def process_image(name, group, x, y):
    # 检查源图片是否存在，避免处理不存在的文件
    source = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', name)
    if not os.path.exists(source):
        # print(f"File not found: {source}")
        return None

    # 分类图像
    classification = classify_image(group)
    
    # 处理这个图片的所有框
    results = []
    for _, row in group.iterrows():
        results.append((
            x,                  # x
            y,                  # y
            row['box_no'],      # box_no
            row['x1'],          # x1
            row['y1'],          # y1
            row['x2'],          # x2
            row['y2'],          # y2
            name,               # Image Name
            classification,     # Classification
            row['uv'],         # uv
            row['click_uv'],   # click_uv
            row['最终分层']     # 添加最终分层
        ))

    # 复制图片到分类文件夹
    destination_dir = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_img_classified_images', classification)
    os.makedirs(destination_dir, exist_ok=True)
    destination = os.path.join(destination_dir, name)
    shutil.copy(source, destination)

    # 绘制并保存线框图
    visualization_dir = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_img_visualizations'
    os.makedirs(visualization_dir, exist_ok=True)
    draw_rectangles(group, os.path.join(visualization_dir, f"{name}_visualization.png"))

    return results

# 主处理函数
def main(x, y):
    print(f"Processing for x: {x}, y: {y}")
    df = pd.read_excel(f'D://code//data//{path}//{z}//merged_info_ctr-1.xlsx')
    
    # 添加验证代码
    required_columns = ['box_no', 'uv', 'click_uv']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"'{col}' 列在Excel文件中不存在")
    
    df['Image Name'] = df['Image Name'].astype(str) + '.jpg'

    # 计算 x2 和 y2
    df['x1'] = df['img_x1']
    df['y1'] = df['img_y1']
    df['x2'] = df['img_x2']
    df['y2'] = df['img_y2']

    # 删除重复 'ctr' 列
    if 'ctr' in df.columns and df.columns.duplicated().any():
        df = df.loc[:, ~df.columns.duplicated()]

    # 按 'ctr' 列排序并取前50%
    df = df.sort_values(by='ctr', ascending=False)
    rows_to_keep = int(len(df) * 0.5)  # 取前50% ###############################################
    df = df.head(rows_to_keep)
    df = df.apply(normalize_coordinates, axis=1)

    # 添加这行：按 'Image Name' 分组
    grouped = df.groupby('Image Name')

    # 按 'Image Name' 分组并处理
    # 修改结果收集方式
    with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = [executor.submit(process_image, name, group, x, y) for name, group in grouped]

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            results = future.result()
            if results is not None:
                for result in results:
                    all_results.append(result)

# 处理所有 x 和 y 组合
for x, y in itertools.product(x_list, y_list):
    main(x, y)

# 保存总表
output_df = pd.DataFrame(all_results, columns=[
    'x', 'y', 'box_no', 'x1', 'y1', 'x2', 'y2', 
    'Image Name', 'Classification', 'uv', 'click_uv', '最终分层'
])
output_file = f'D://code//data//{path}//{z}//img_all_classification_results-1.xlsx'
output_df.to_excel(output_file, index=False)

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")
print(output_file)




# step6.1 - 生成基于商品主体数量的饼图和excel
## 不分brand,all in one

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

# 读取数据表
file_path = f'D://code//data//{path}//{z}//img_all_classification_results-1.xlsx'
df = pd.read_excel(file_path)
df = df.dropna()  # 去除空值

# 设置保存图像的文件夹
output_folder = f'D://code//data//{path}//{z}//url_3'
os.makedirs(output_folder, exist_ok=True)

# 现代时尚的配色方案
modern_colors = [
    '#FF6B6B',  # 珊瑚红
    '#4ECDC4',  # 青绿
    '#45B7D1',  # 天蓝
    '#96CEB4',  # 薄荷绿
    '#FFEEAD',  # 淡黄
]

# 创建一个列表来存储表格数据
table_data = []

# 1. 饼图部分和数据收集
grouped_by_xy = df.groupby(['x', 'y'])

for (x, y), group in grouped_by_xy:
    # 计算每个box_no的统计信息
    box_stats = group.groupby('box_no').agg({
        'uv': 'sum',
        'click_uv': 'sum'
    }).reset_index()
    
    # 计算CTR和添加count列
    box_stats['ctr'] = box_stats['click_uv'] / box_stats['uv']
    box_stats['count'] = group['box_no'].value_counts().reindex(box_stats['box_no']).values
    total_count = box_stats['count'].sum()
    box_stats['count_pct'] = box_stats['count'] / total_count  # 添加count百分比列
    
    # 添加到表格数据
    for _, row in box_stats.iterrows():
        table_data.append({
            'x': x,
            'y': y,
            'box_no': row['box_no'],
            'brand': 'all',
            'uv': row['uv'],
            'click_uv': row['click_uv'],
            'ctr': row['ctr'],
            'count': row['count'],
            'count_pct': row['count_pct']  # 百分比列
        })
    
    # 绘制饼图
    box_no_counts = group['box_no'].value_counts()
    labels = [f'No:{idx}\nCount: {box_no_counts[idx]}\nCTR: {box_stats[box_stats["box_no"] == idx]["ctr"].values[0]:.2%}' 
            for idx in box_no_counts.index]

    plt.figure(figsize=(8, 8))
    plt.pie(box_no_counts, 
            labels=labels, 
            autopct='%1.1f%%', 
            startangle=140, 
            colors=modern_colors,
            shadow=False,
            textprops={'fontsize': 12})
    plt.title(f'{x}_{y} Box Distribution', pad=20, fontsize=14)
    
    pie_path = os.path.join(output_folder, f'{x}_{y}_img_url_3_boxper.png')
    plt.savefig(pie_path, bbox_inches='tight', dpi=100)
    print(f"饼图已生成并保存: {pie_path}")  # 打印饼图路径
    plt.close()

# 创建并保存表格
table_df = pd.DataFrame(table_data)
table_df = table_df[['x', 'y', 'box_no', 'brand', 'uv', 'click_uv', 'ctr', 'count', 'count_pct']]  # 确保列的顺序
table_df.sort_values(['x', 'y', 'box_no'], inplace=True)  # 排序
table_output_path = os.path.join(output_folder, 'img_all_url_3_boxper.xlsx')
table_df.to_excel(table_output_path, index=False)
print(f"Excel表格已保存: {table_output_path}")  # 打印Excel路径

# 2. 热力图部分
grouped_by_xyz = df.groupby(['x', 'y', 'box_no'])

for (x, y, box_no), group in grouped_by_xyz:
    heatmap_data = np.zeros((800, 800))

    for _, row in group.iterrows():
        scale_factor = 800 / max(800, row['x2'], row['y2'])
        x1 = int(row['x1'] * scale_factor)
        y1 = int(row['y1'] * scale_factor)
        x2 = int(row['x2'] * scale_factor)
        y2 = int(row['y2'] * scale_factor)
        
        x1, x2 = max(0, min(x1, 799)), max(0, min(x2, 799))
        y1, y2 = max(0, min(y1, 799)), max(0, min(y2, 799))
        heatmap_data[y1:y2, x1:x2] += 1

    plt.figure(figsize=(8, 8))
    sns.heatmap(heatmap_data, 
                cmap='YlOrRd',
                cbar=False,
                xticklabels=False,
                yticklabels=False)
    
    heatmap_path = os.path.join(output_folder, f'{x}_{y}_img_boxno_{box_no}_brand_all_url_3_boxhot.png')
    plt.title(f'{x}_{y}_{box_no} Heatmap', pad=20, fontsize=14)
    plt.savefig(heatmap_path, bbox_inches='tight', dpi=100, pad_inches=0)
    print(f"热力图已生成并保存: {heatmap_path}")  # 打印热力图路径
    plt.close()

print("所有图表和统计表格已生成并保存。")

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")
print(f"All processing completed at: {formatted_time}")
print(f"All processing completed at: {formatted_time}")





In [ ]:
# import pandas as pd
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import datetime

# # 定义 x_list 和 y_list
# x_list = ['9736', '9735', '12004']
# y_list = ['txt', 'price']
# path = 'Lv2期结论'
# z = '男士春夏下装_from_0501'

# # 读取数据表
# file_path = f'D://code//data//{path}//{z}//img_all_classification_results-1.xlsx'
# df = pd.read_excel(file_path)
# df = df.dropna()  # 去除空值

# # 设置保存图像的文件夹
# output_folder = f'D://code//data//{path}//{z}//url_3'
# os.makedirs(output_folder, exist_ok=True)

# # 现代时尚的配色方案
# modern_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEEAD']

# # 创建一个列表来存储表格数据
# table_data = []

# # 确保 x 和 y 类型一致
# df['x'] = df['x'].astype(str)  # 或根据实际情况改为 int
# df['y'] = df['y'].astype(str)

# for x in x_list:
#     for y in y_list:
#         # 1. 饼图部分和数据收集
#         grouped_by_xy = df[(df['x'] == x) & (df['y'] == y)].groupby(['x', 'y'])

#         for (x, y), group in grouped_by_xy:
#             # 计算每个 box_no 的统计信息
#             box_stats = group.groupby('box_no').agg({
#                 'uv': 'sum',
#                 'click_uv': 'sum'
#             }).reset_index()
            
#             # 计算 CTR 和添加 count 列
#             box_stats['ctr'] = box_stats['click_uv'] / box_stats['uv']
#             box_stats['count'] = group['box_no'].value_counts().reindex(box_stats['box_no']).values
#             total_count = box_stats['count'].sum()
#             box_stats['count_pct'] = box_stats['count'] / total_count  # 添加 count 百分比列
            
#             # 添加到表格数据
#             for _, row in box_stats.iterrows():
#                 row_data = {
#                     'x': x,
#                     'y': y,
#                     'box_no': row['box_no'],
#                     'uv': row['uv'],
#                     'click_uv': row['click_uv'],
#                     'ctr': row['ctr'],
#                     'count': row['count'],
#                     'count_pct': row['count_pct']  # 百分比列
#                 }
#                 table_data.append(row_data)
            
#             # 绘制饼图
#             box_no_counts = group['box_no'].value_counts()
#             labels = [f'No:{idx}\nCount: {box_no_counts[idx]}\nCTR: {box_stats[box_stats["box_no"] == idx]["ctr"].values[0]:.2%}' 
#                     for idx in box_no_counts.index]

#             plt.figure(figsize=(8, 8))
#             plt.pie(box_no_counts, 
#                     labels=labels, 
#                     autopct='%1.1f%%', 
#                     startangle=140, 
#                     colors=modern_colors,
#                     shadow=False,
#                     textprops={'fontsize': 12})
#             plt.title(f'{x}_{y} Box Distribution', pad=20, fontsize=14)
#             pie_path = os.path.join(output_folder, f'{x}_{y}_img_url_3_boxper.png')
#             plt.savefig(pie_path, bbox_inches='tight', dpi=100)
#             print(f"饼图已生成并保存: {pie_path}")  # 打印饼图路径
#             plt.close()

# # 创建并保存表格
# table_df = pd.DataFrame(table_data)
# table_df = table_df[['x', 'y', 'box_no', 'uv', 'click_uv', 'ctr', 'count', 'count_pct']]  # 确保列的顺序
# table_df.sort_values(['x', 'y', 'box_no'], inplace=True)  # 排序
# table_output_path = os.path.join(output_folder, 'img_all_url_3_boxper.xlsx')
# table_df.to_excel(table_output_path, index=False)
# print(f"Excel表格已保存: {table_output_path}")  # 打印Excel路径

# # 打印当前时间
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(f"All processing completed at: {formatted_time}")




# step6.2 - 生成基于商品主体数量的饼图和excel
### 分brand, 类似 filter_layer_cases = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

In [ ]:
# import pandas as pd
# import os
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt


# # 读取 Excel 文件
# df = pd.read_excel(f'D://code//data//{path}//{z}//all_classification_results-1.xlsx')

# # 创建字典，用于存储每个 "Image Name" 的非空 "最终分层" 值
# final_layer_map = df.dropna(subset=['最终分层']).set_index('Image Name')['最终分层'].to_dict()

# # 定义填充函数
# def fill_final_layer(row):
#     if pd.isna(row['最终分层']):  # 检查 "最终分层" 是否为空
#         return final_layer_map.get(row['Image Name'], row['最终分层'])  # 使用字典中的值填充
#     return row['最终分层']

# # 应用填充函数
# df['最终分层'] = df.apply(fill_final_layer, axis=1)

# # 过滤层级
# output_folder = f'D://code//data//{path}//{z}//url_3'
# os.makedirs(output_folder, exist_ok=True)  # 创建主输出文件夹

# # 存储统计结果的列表
# summary_data = []

# # 遍历 filter_layer_cases 中的每个子列表
# for layers in filter_layer_cases:
#     # 筛选符合当前 layers 的数据
#     filtered_df = df[df['最终分层'].isin(layers)]
    
#     # 将 layers 转换为字符串格式 1.0_2.0_3.0
#     layer_suffix = "_".join(map(str, layers))

#     # 按 x, y, box_no 进行分组
#     grouped_by_xyz = filtered_df.groupby(['x', 'y', 'box_no'])

#     for (x, y, box_no), group in grouped_by_xyz:
#         # 计算 uv、click_uv 和 ctr
#         uv = group['uv'].sum()
#         click_uv = group['click_uv'].sum()
#         ctr = click_uv / uv if uv != 0 else 0  # 避免除以零

#         # 将统计结果添加到 summary_data 列表中
#         summary_data.append([x, y, box_no, layer_suffix, uv, click_uv, ctr])

#         # 创建热力图数据
#         heatmap_data = np.zeros((800, 800))

#         for _, row in group.iterrows():
#             # 计算缩放比例
#             scale_factor = 800 / max(800, row['x2'], row['y2'])
#             x1 = int(row['x1'] * scale_factor)
#             y1 = int(row['y1'] * scale_factor)
#             x2 = int(row['x2'] * scale_factor)
#             y2 = int(row['y2'] * scale_factor)
            
#             # 限制在画布范围内
#             x1, x2 = max(0, min(x1, 799)), max(0, min(x2, 799))
#             y1, y2 = max(0, min(y1, 799)), max(0, min(y2, 799))
            
#             # 累加区域热度
#             heatmap_data[y1:y2, x1:x2] += 1

#         # 绘制热力图
#         plt.figure(figsize=(8, 8))
#         sns.heatmap(heatmap_data, 
#                     cmap='YlOrRd',
#                     cbar=False,
#                     xticklabels=False,
#                     yticklabels=False)
        
#         plt.title(f'{x}_{y}_{box_no} Heatmap', pad=20, fontsize=14)
        
#         # 保存热力图
#         heatmap_path = os.path.join(output_folder, f'{x}_{y}_boxno_{box_no}_brand_{layer_suffix}_url_3_boxhot.png')
#         plt.savefig(heatmap_path, bbox_inches='tight', dpi=100, pad_inches=0)
#         plt.close()

# # 将统计结果保存为 Excel 文件
# summary_df = pd.DataFrame(summary_data, columns=['x', 'y', 'box_no', 'filter_layer_cases', 'uv', 'click_uv', 'ctr'])
# summary_file = os.path.join(output_folder, f'{x}_{y}_brand_{layers}_url_3_boxper.xlsx')
# summary_df.to_excel(summary_file, index=False)

# print("所有热力图和统计数据已生成并保存到指定文件夹。")




In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

# # 定义 x_list 和 y_list
# x_list = ['9736', '9735', '12004']
# y_list = ['txt', 'price']
# path = 'Lv2期结论'
# z = '男士春夏下装_from_0501'

# # 定义过滤层级条件
# filter_layer_cases = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]  # 示例分层条件

# 读取 Excel 文件
df = pd.read_excel(f'D://code//data//{path}//{z}//img_all_classification_results-1.xlsx')

# 填充空的“最终分层”列
final_layer_map = df.dropna(subset=['最终分层']).set_index('Image Name')['最终分层'].to_dict()
df['最终分层'] = df.apply(lambda row: final_layer_map.get(row['Image Name'], row['最终分层']), axis=1)

output_folder = f'D://code//data//{path}//{z}//url_3'
os.makedirs(output_folder, exist_ok=True)

# 创建一个总汇总表的 DataFrame
all_summary_data = []

# 遍历 x_list 和 y_list 的每个组合
for x in x_list:
    for y in y_list:
        # 遍历 filter_layer_cases 中的每个子列表
        for layers in filter_layer_cases:
            summary_data = []  # 每个 layers 的组合重置 summary_data
            layer_suffix = "_".join(map(str, layers))

            # 筛选符合当前 x, y, 和 layers 的数据
            filtered_df = df[(df['x'] == int(x)) & (df['y'] == y) & (df['最终分层'].isin(layers))]

            grouped_by_xyz = filtered_df.groupby(['x', 'y', 'box_no'])
            for (x_val, y_val, box_no), group in grouped_by_xyz:
                uv = group['uv'].sum()
                click_uv = group['click_uv'].sum()
                ctr = click_uv / uv if uv != 0 else 0
                count = len(group)
                count_pct = count / len(filtered_df) if len(filtered_df) > 0 else 0

                summary_data.append([x_val, y_val, box_no, layer_suffix, uv, click_uv, ctr, count, count_pct])

                # 创建热力图
                heatmap_data = np.zeros((800, 800))
                for _, row in group.iterrows():
                    scale_factor = 800 / max(800, row['x2'], row['y2'])
                    x1 = int(row['x1'] * scale_factor)
                    y1 = int(row['y1'] * scale_factor)
                    x2 = int(row['x2'] * scale_factor)
                    y2 = int(row['y2'] * scale_factor)

                    x1, x2 = max(0, min(x1, 799)), max(0, min(x2, 799))
                    y1, y2 = max(0, min(y1, 799)), max(0, min(y2, 799))
                    heatmap_data[y1:y2, x1:x2] += 1

                plt.figure(figsize=(8, 8))
                sns.heatmap(heatmap_data, cmap='YlOrRd', cbar=False, xticklabels=False, yticklabels=False)
                plt.title(f'{x_val}_{y_val}_{box_no} Heatmap - {layer_suffix}', pad=20, fontsize=14)

                heatmap_path = os.path.join(output_folder, f'{x_val}_{y_val}_img_boxno_{box_no}_brand_{layer_suffix}_url_3_boxhot.png')
                plt.savefig(heatmap_path, bbox_inches='tight', dpi=100, pad_inches=0)
                plt.close()
                print(f"Generated heatmap: {heatmap_path}")

            # 将 summary_data 转换为 DataFrame
            summary_df = pd.DataFrame(summary_data, columns=['x', 'y', 'box_no', 'brand', 'uv', 'click_uv', 'ctr', 'count', 'count_pct'])
            
            # 将 summary_df 数据追加到汇总表中
            all_summary_data.append(summary_df)

            # 绘制饼图
            for (x_val, y_val), group in summary_df.groupby(['x', 'y']):
                labels = [f'Box {int(row["box_no"])}\nCount: {row["count"]}\nCTR: {row["ctr"]:.2%}' for _, row in group.iterrows()]
                
                plt.figure(figsize=(8, 8))
                plt.pie(group['count'], labels=labels, autopct='%1.1f%%', startangle=140)
                plt.title(f'{x_val}_{y_val} Box Distribution - {layer_suffix}', pad=20, fontsize=14)
                pie_chart_path = os.path.join(output_folder, f'{x_val}_{y_val}_img_brand_{layer_suffix}_url_3_boxper.png')
                plt.savefig(pie_chart_path, bbox_inches='tight', dpi=100)
                plt.close()
                print(f"Generated pie chart: {pie_chart_path}")

# 将所有的 summary_df 汇总到一个 DataFrame
combined_summary_df = pd.concat(all_summary_data, ignore_index=True)

# 保存汇总数据到单一 Excel 文件
combined_summary_file = os.path.join(output_folder, 'img_brand_all_url_3_boxper.xlsx')
combined_summary_df.to_excel(combined_summary_file, index=False)
print(f"汇总 Excel 文件已保存到: {combined_summary_file}")

print("所有热力图、饼图和统计数据已生成并保存到指定文件夹。")

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")



file1_path = os.path.join(output_folder, 'img_all_url_3_boxper.xlsx')
file2_path = os.path.join(output_folder, 'img_brand_all_url_3_boxper.xlsx')

# 读取两个Excel文件
df1 = pd.read_excel(file1_path)
df2 = pd.read_excel(file2_path)

# 将两个DataFrame上下拼接
combined_df = pd.concat([df1, df2], ignore_index=True)

# 保存合并后的文件
combined_file_path = os.path.join(output_folder, 'img_url_3_boxper.xlsx')  # 最终数据
combined_df.to_excel(combined_file_path, index=False)

print(f"合并后的Excel文件已保存到: {combined_file_path}")

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")
print(f"All processing completed at: {formatted_time}")
print(f"All processing completed at: {formatted_time}")



# step7 - 文本:布局分类&热力图生成
### 重新汇总了一个merged_info_ctr-123,并生成了布局分类的热力图,核心是和之前的数据格式不一样

In [ ]:
# 这里重新汇总了一个merged_info_ctr-123


import pandas as pd
import cv2
import numpy as np
from PIL import Image
import os



def merge_excel_files(txt_box_info_file, img_box_info_file, output_file):
    # 读取 txt_box_info 文件
    txt_df = pd.read_excel(txt_box_info_file)
    txt_df = txt_df.rename(columns={'File Name':'Image Name', 'x1': 'txt_x1', 'y1': 'txt_y1', 'x2': 'txt_x2', 'y2': 'txt_y2',})

    # 读取 img_box_info 文件
    img_df = pd.read_excel(img_box_info_file)

    # 重命名列
    img_df = img_df.rename(columns={'x1': 'img_x1', 'y1': 'img_y1', 'x3': 'img_x2', 'y3':'img_y2', 'Subfolder':'Style'})
    img_df = img_df.loc[:, ['Image Name', 'Style','img_x1', 'img_y1', 'img_x2', 'img_y2', 'box_no', ]]

    img_df = img_df.drop_duplicates()
    
    # 合并两个 DataFrame，使用 txt_box_info 的表头作为准
    merged_df = pd.concat([txt_df, img_df], ignore_index=True)
    
    # 将 img_box_info 中缺少的数据设置为空
    merged_df = merged_df.fillna("")

    # 使用正则表达式删除 .jpg 或 .png 后缀
    merged_df['Image Name'] = merged_df['Image Name'].str.replace(r'\.(?:jpg|png)$', '', regex=True)
    
    # 将合并后的 DataFrame 写入新的 Excel 文件
    with pd.ExcelWriter(output_file) as writer:
        merged_df.to_excel(writer, index=False)


if __name__ == '__main__':
    txt_box_info_file = f"D://code//data//Lv2期结论//{z}//txt_info-1.xlsx"
    img_box_info_file = f"D://code//data//Lv2期结论//{z}//img_info.xlsx"
    output_file = f"D://code//data//Lv2期结论//{z}//merged_info-123.xlsx"
    
    merge_excel_files(txt_box_info_file, img_box_info_file, output_file)
    
    print("Excel files merged successfully!")




import os
import pandas as pd
import re

# 定义路径
data_1 = f"D://code//data//Lv2期结论//{z}//merged_info-123.xlsx"
data_2 = f"D://code//data//Lv2期结论//{z}//{z}.csv"
# data_3 = 'D://code//data//howtodo_from_0401//服饰鞋靴箱包//品类聚类-服饰鞋靴箱包.csv'
output_path = f"D://code//data//Lv2期结论//{z}//merged_info_ctr-123.xlsx"

# 读取df1
df1 = pd.read_excel(data_1)

# 读取df2
df2 = pd.read_csv(data_2)

aggregated_data = df2.groupby('img_url').agg({
    'cid2': 'first',  # 使用 'first' 函数来选择分组中的第一个值
    'cid3': 'first',
    'uv': 'sum',
    'click_uv': 'sum',
    'gmv_cj':'sum',
    'sale_qtty_cj':'sum'
#     'folder_path': 'first'  # 同样使用 'first' 函数选择第一个值
}).reset_index()  # 重置索引

df2 = aggregated_data

# 计算ctr字段
df2['ctr'] = df2['click_uv'] / df2['uv']


def extract_filename(x):
    # 分割路径，取倒数第二部分和最后一部分（文件名部分）
    parts = x.split('/')
    return f"{parts[-2]}_{os.path.splitext(parts[-1])[0]}"  # 保留原文件扩展名
    # return f"{os.path.splitext(parts[-1])[0]}.jpg"  # 保留原文件扩展名

# 应用函数
df2['only_2'] = df2['img_url'].apply(extract_filename)

# 初始化结果列表
results = []

# 遍历df1的每一行
for index, row1 in df1.iterrows():
    # 查找df2中匹配的行
    matching_rows_df2 = df2[df2['only_2'] == row1['Image Name']]
    
    # 如果没有找到匹配的行，则只添加df1的当前行
    if matching_rows_df2.empty:
        results.append(row1.to_dict())
    else:
        # 对于找到的每个匹配行，先添加df1的当前行，然后添加匹配的df2行
        results.append(row1.to_dict())
        for _, row2 in matching_rows_df2.iterrows():
            # 可能需要添加额外的逻辑来处理多个匹配的情况
            # 这里假设每个df1的行在df2中最多只有一个匹配
            merged_row = {**row1.to_dict(), **row2.to_dict()}
            results.append(merged_row)

# 将结果列表转换为DataFrame
result_df = pd.DataFrame(results)

result_df_drop = result_df.dropna(subset=['uv'])

# 保存到指定路径
result_df_drop.to_excel(output_path, index=False)

print(f"Merged file saved to {output_path}")

import datetime

current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

print(formatted_time)
print(formatted_time)
print(formatted_time)




# step7.1 - 文本:布局分类
### 不包含brand分类, all in one

In [ ]:
# 核心功能: ①从图片中提取文本框,形成线框图; ②将线框图按照九宫格进行分类
# 这里是把x和y都采用了list的形式,用来简化人力的输入
# 这里是针对整体,没有分structure,也没有分brand
# 在这一步拼了一张"structure-映射表",用来把人工分类加入进来
#



import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import shutil
import os
from tqdm import tqdm
import concurrent.futures
import datetime
import numpy as np
import seaborn as sns
import itertools



# # 定义 x_list 和 y_list（只需要一次）
# # x_list = ['6913','6912','6911','6910','6909','6908','12066']  # 添加所有需要的 Subfolder 值
# x_list = ['9736','9735','12004']  # 添加所有需要的 Subfolder 值
# y_list = ['txt', 'price']    # 添加所有需要的 Style 值
# path = 'Lv2期结论'


# z = '男士春夏下装_from_0501'



# 在文件开头定义一个函数来读取原始数据
def read_original_data(z):
    file_path = os.path.join(f'D://code//data//{path}//{z}//merged_info_ctr-123.xlsx')
    # print(f"Reading data from: {file_path}")
    df = pd.read_excel(file_path)
    df['Image Name'] = df['Image Name'] + '.jpg'
    # print(f"Read {df.shape[0]} rows and {df.shape[1]} columns")
    return df

# 1. 读取和预处理数据
def normalize_coordinates(row):
    width = 616
    height = 616
    row['left_norm'] = max(0, min(row['txt_x1'] / width, 1))
    row['top_norm'] = max(0, min(row['txt_y1'] / height, 1))
    row['right_norm'] = max(0, min(row['txt_x2'] / width, 1))
    row['bottom_norm'] = max(0, min(row['txt_y2'] / height, 1))
    return row

# 2. 绘制矩形和网格
def draw_rectangles(group):
    fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
    
    for _, row in group.iterrows():
        rect = Rectangle((row['left_norm'], 1 - row['bottom_norm']), 
                         row['right_norm'] - row['left_norm'], 
                         row['bottom_norm'] - row['top_norm'],
                         fill=False, edgecolor='r')
        ax.add_patch(rect)
    
    for i in range(3):
        for j in range(3):
            rect = Rectangle((j/3, 1 - (i+1)/3), 1/3, 1/3, fill=False, edgecolor='b')
            ax.add_patch(rect)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    return fig, ax

# 3. 判断重叠和分类
def check_overlap(rect, grid_cell):
    return not (rect[2] < grid_cell[0] or rect[0] > grid_cell[2] or
                rect[3] < grid_cell[1] or rect[1] > grid_cell[3])

def classify_image(group):
    overlaps = [0] * 9
    
    for _, row in group.iterrows():
        rect = (row['left_norm'], row['top_norm'], row['right_norm'], row['bottom_norm'])
        for i in range(3):
            for j in range(3):
                grid_cell = (j/3, i/3, (j+1)/3, (i+1)/3)
                if check_overlap(rect, grid_cell):
                    overlaps[i*3 + j] = 1
    return ''.join(map(str, overlaps))  # 直接使用join方法生成字符串

# 4. 处理单个图像
def process_image(name, group, x, y):
    try:
        classification = classify_image(group)
        
        # 复制图片
        source = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', name)
        destination = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classified_images', classification.zfill(9), name)
        os.makedirs(os.path.dirname(destination), exist_ok=True)
        shutil.copy2(source, destination)
        
        # 保存可视化结果
        fig, ax = draw_rectangles(group)
        visualization_name = f"{name.split('.')[0]}_visualization.png"
        fig.savefig(os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_visualizations', visualization_name), bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        
        return name, classification
    except Exception as e:
        # print(f"Error processing image {name}: {str(e)}")
        return name, None

# 主处理函数
def main(x, y, original_df):
    print(f"Processing for Subfolder={x}, Style={y}...")
    print("读取并预处理数据...")

    # 使用原始数据的副本
    df = original_df.copy()

    # print(f"Original columns: {df.columns.tolist()}")
    print(f"Original shape: {df.shape}")

    # 在读取df时添加筛选条件
    def filter_by_rectangle(row):
        right, bottom = 616 * 0.3, 616 * 0.2
        if row['txt_x2'] < right and row['txt_y2'] < bottom:
            return False
        return True

    # 修改数据筛选逻辑
    df = df[(df['img_x1'].isna()) & (df['Subfolder'] == int(x)) & (df['Style'] == y) & (df['Type'] == 'Original')]
    df = df.sort_values('ctr', ascending=False)

    # 应用左上角的筛选条件
    df = df[df.apply(filter_by_rectangle, axis=1)]

    rows_to_keep = int(len(df) * 0.5)  # 取前50%的样本 ##################################################
    df = df.head(rows_to_keep)

    # 应用normalize_coordinates函数
    df = df.apply(normalize_coordinates, axis=1)

    # 检查是否存在所需的列
    required_columns = ['left_norm', 'top_norm', 'right_norm', 'bottom_norm']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"Error: Missing columns: {missing_columns}")
        return

    # 创建输出目录
    classified_images_dir = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', '50%_word_classified_images')
    visualizations_dir = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', '50%_word_visualizations')
    os.makedirs(classified_images_dir, exist_ok=True)
    os.makedirs(visualizations_dir, exist_ok=True)

    # 按File name分组并处理
    grouped = df.groupby('Image Name')
    print("处理图像...")

    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = [executor.submit(process_image, name, group, x, y) for name, group in grouped]
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            name, classification = future.result()
            results.append((name, classification))
    
    # 在原Excel文件中新增一列，保存分类结果
    classification_df = pd.DataFrame(results, columns=['Image Name', 'Classification'])
    df = pd.merge(df, classification_df, on='Image Name', how='left')
    df.to_excel(os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification-{x}_{y}.xlsx'), index=False)
    
    # 读取Excel
    df_maping = pd.read_excel(os.path.join(f'D://code//data//{path}//structure-映射表.xlsx'))
    df_maping['Classification'] = df_maping['Classification'].astype(str).str.zfill(9)
    # 当df_maping的Classification列的值等于df中Classification列的值, 将df_maping的structure列的值添加到df对应行之后
    df = pd.merge(df, df_maping, left_on='Classification', right_on='Classification', how='left')
    df.to_excel(os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}.xlsx'), index=False)

    print(f"Completed processing for Subfolder={x}, Style={y}")
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print("--------------------")

if __name__ == "__main__":
    # 在循环开始前读取原始数据
    original_df = read_original_data(z)
    
    for x, y in itertools.product(x_list, y_list):
        main(x, y, original_df)

import datetime

current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

print(formatted_time)
print(formatted_time)
print(formatted_time)




# step7.2 - 文本:布局分类
### 包含brand分类


In [ ]:
# 核心功能: ①从图片中提取文本框,形成线框图; ②将线框图按照九宫格进行分类
# 这里是把x和y都采用了list的形式,用来简化人力的输入
# 这里添加了brand的分类
# 在这一步拼了一张"structure-映射表",用来把人工分类加入进来
#



import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import shutil
import os
from tqdm import tqdm
import concurrent.futures
import datetime
import numpy as np
import seaborn as sns
import itertools


# # 定义筛选条件
# filter_layer_cases = [[1.0, 2.0, 3.0],[4.0, 5.0, 6.0]]



for filter_layers in filter_layer_cases:
    # 在文件开头定义一个函数来读取原始数据
    def read_original_data(z):
        file_path = os.path.join(f'D://code//data//{path}//{z}//merged_info_ctr-123.xlsx')
        df = pd.read_excel(file_path)
        df['Image Name'] = df['Image Name'] + '.jpg'
        return df

    # 1. 读取和预处理数据
    def normalize_coordinates(row):
        width = 616
        height = 616
        row['left_norm'] = max(0, min(row['txt_x1'] / width, 1))
        row['top_norm'] = max(0, min(row['txt_y1'] / height, 1))
        row['right_norm'] = max(0, min(row['txt_x2'] / width, 1))
        row['bottom_norm'] = max(0, min(row['txt_y2'] / height, 1))
        return row

    # 2. 绘制矩形和网格
    def draw_rectangles(group):
        fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
        
        for _, row in group.iterrows():
            rect = Rectangle((row['left_norm'], 1 - row['bottom_norm']), 
                            row['right_norm'] - row['left_norm'], 
                            row['bottom_norm'] - row['top_norm'],
                            fill=False, edgecolor='r')
            ax.add_patch(rect)
        
        for i in range(3):
            for j in range(3):
                rect = Rectangle((j/3, 1 - (i+1)/3), 1/3, 1/3, fill=False, edgecolor='b')
                ax.add_patch(rect)
        
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')
        return fig, ax

    # 3. 判断重叠和分类
    def check_overlap(rect, grid_cell):
        return not (rect[2] < grid_cell[0] or rect[0] > grid_cell[2] or
                    rect[3] < grid_cell[1] or rect[1] > grid_cell[3])

    def classify_image(group):
        overlaps = [0] * 9
        
        for _, row in group.iterrows():
            rect = (row['left_norm'], row['top_norm'], row['right_norm'], row['bottom_norm'])
            for i in range(3):
                for j in range(3):
                    grid_cell = (j/3, i/3, (j+1)/3, (i+1)/3)
                    if check_overlap(rect, grid_cell):
                        overlaps[i*3 + j] = 1
        return ''.join(map(str, overlaps))  # 直接使用join方法生成字符串

    # 4. 处理单个图像
    def process_image(name, group, x, y, filter_suffix):
        try:
            classification = classify_image(group)
            
            # 复制图片
            source = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', name)
            destination = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classified_images_{filter_suffix}', classification.zfill(9), name)
            os.makedirs(os.path.dirname(destination), exist_ok=True)
            shutil.copy2(source, destination)
            
            # 保存可视化结果
            fig, ax = draw_rectangles(group)
            visualization_name = f"{name.split('.')[0]}_visualization.png"
            fig.savefig(os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_visualizations_{filter_suffix}', visualization_name), bbox_inches='tight', pad_inches=0)
            plt.close(fig)
            
            return name, classification
        except Exception as e:
            return name, None

    # 主处理函数
    def main(x, y, original_df, filter_layers):
        filter_suffix = f"filter_{'_'.join(map(str, filter_layers))}"
        print(f"Processing for Subfolder={x}, Style={y}, Filter={filter_suffix}...")
        print("读取并预处理数据...")

        # 使用原始数据的副本
        df = original_df.copy()

        print(f"Original shape: {df.shape}")

        # 在读取df时添加筛选条件
        def filter_by_rectangle(row):
            right, bottom = 616 * 0.3, 616 * 0.2
            if row['txt_x2'] < right and row['txt_y2'] < bottom:
                return False
            return True

        # 修改数据筛选逻辑
        df = df[(df['img_x1'].isna()) & (df['Subfolder'] == int(x)) & (df['Style'] == y) & (df['Type'] == 'Original') & (df['最终分层'].isin(filter_layers))]
        df = df.sort_values('ctr', ascending=False)

        # 应用左上角的筛选条件
        df = df[df.apply(filter_by_rectangle, axis=1)]

        rows_to_keep = int(len(df) * 0.5)  # 取前50%的样本 ##################################################
        df = df.head(rows_to_keep)

        # 应用normalize_coordinates函数
        df = df.apply(normalize_coordinates, axis=1)

        # 检查是否存在所需的列
        required_columns = ['left_norm', 'top_norm', 'right_norm', 'bottom_norm']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Error: Missing columns: {missing_columns}")
            return

        # 创建输出目录
        classified_images_dir = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', f'50%_word_classified_images_{filter_suffix}')
        visualizations_dir = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', f'50%_word_visualizations_{filter_suffix}')
        os.makedirs(classified_images_dir, exist_ok=True)
        os.makedirs(visualizations_dir, exist_ok=True)

        # 按File name分组并处理
        grouped = df.groupby('Image Name')
        print("处理图像...")

        results = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            futures = [executor.submit(process_image, name, group, x, y, filter_suffix) for name, group in grouped]
            
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
                name, classification = future.result()
                results.append((name, classification))
        
        # 在原Excel文件中新增一列，保存分类结果
        classification_df = pd.DataFrame(results, columns=['Image Name', 'Classification'])
        df = pd.merge(df, classification_df, on='Image Name', how='left')
        df.to_excel(os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification-{x}_{y}_{filter_suffix}.xlsx'), index=False)

        # 读取Excel
        df_maping = pd.read_excel(os.path.join(f'D://code//data//{path}//structure-映射表.xlsx'))
        df_maping['Classification'] = df_maping['Classification'].astype(str).str.zfill(9)
        # 当df_maping的Classification列的值等于df中Classification列的值, 将df_maping的structure列的值添加到df对应行之后
        df = pd.merge(df, df_maping, left_on='Classification', right_on='Classification', how='left')
        df.to_excel(os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}_{filter_suffix}.xlsx'), index=False)
        
        print(f"Completed processing for Subfolder={x}, Style={y}, Filter={filter_suffix}")
        print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print("--------------------")

    if __name__ == "__main__":
        # 在循环开始前读取原始数据
        original_df = read_original_data(z)
        
        for x, y in itertools.product(x_list, y_list):
            main(x, y, original_df, filter_layers)

import datetime

current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

print(formatted_time)
print(formatted_time)
print(formatted_time)




# step7.3 - 文本:分类后的热力图生成
### 没有按照structure进行分类, 是将classification全量进行输出
### 没有针对brand进行分类

In [ ]:
# 针对分类结果,绘制文本框热力图,但是通过x和y的list来实现


import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import datetime



# 创建保存结果的文件夹
def create_output_folder(x, y):
    output_folder = os.path.join(f'D://code//data//{path}//{z}')
    os.makedirs(output_folder, exist_ok=True)

    # 创建 url_2 子文件夹
    url_2_folder = os.path.join(output_folder, 'url_2')
    os.makedirs(url_2_folder, exist_ok=True)
    return output_folder


# 读取Excel文件
def read_excel_file(x, y):
    file_path = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', f'50%_word_classification-{x}_{y}.xlsx')
    return pd.read_excel(file_path)

# 绘制热力图
def plot_heatmaps(df, output_folder):
    # 按Classification列进行分组
    grouped = df.groupby('Classification')

    # 遍历每个分组
    for name, group in grouped:
        # 创建一个新的图形
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        
        # 绘制矩形框
        ax1.set_xlim(0, 616)
        ax1.set_ylim(616, 0)
        for _, row in group.iterrows():
            x1, y1, x2, y2 = row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']
            # 排除左上角的文本框
            if x1 <= 616 * 0.3 and y1 <= 616 * 0.2:
                continue
            width = x2 - x1
            height = y2 - y1
            rect = plt.Rectangle((x1, y1), width, height, fill=False, edgecolor='r')
            ax1.add_patch(rect)
        ax1.set_title(f'Bounding Boxes for {name}')
        ax1.set_xlabel('X')
        ax1.set_ylabel('Y')

        # 创建热力图
        heatmap = np.zeros((616, 616))
        for _, row in group.iterrows():
            x1, y1, x2, y2 = row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']
            # 排除左上角的文本框
            if x1 <= 616 * 0.3 and y1 <= 616 * 0.2:
                continue
            x1, y1 = max(0, min(x1, 615)), max(0, min(y1, 615))
            x2, y2 = max(0, min(x2, 615)), max(0, min(y2, 615))
            heatmap[int(y1):int(y2)+1, int(x1):int(x2)+1] += 1

        # 绘制热力图
        sns.heatmap(heatmap, ax=ax2, cmap='YlOrRd', cbar=True)
        ax2.set_title(f'Heatmap for {name}')
        ax2.set_xlabel('X')
        ax2.set_ylabel('Y')

        # 调整子图之间的间距
        plt.tight_layout()

        # 保存图像
        plt.savefig(os.path.join(output_folder, f'url_2//{x}_{y}_word_classification_{name}_brand_all_url_2_hotlayout.png'), dpi=100, bbox_inches='tight')
        plt.close()  # 关闭图形，释放内存

        print(f"已保存 {name} 的热力图")

    print("所有热力图已保存在 output_heatmaps 文件夹中")

# 主函数
if __name__ == "__main__":
    for x in x_list:
        for y in y_list:
            print(f"Processing for x={x}, y={y}...")
            output_folder = create_output_folder(x, y)
            file_path = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', f'50%_word_classification-{x}_{y}.xlsx')
            if not os.path.exists(file_path):
                print(f"No data found for x={x}, y={y}. Skipping...")
                continue
            df = read_excel_file(x, y)
            plot_heatmaps(df, output_folder)

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    print(formatted_time)
    print(formatted_time)
    print(formatted_time)




# step7.3 - 文本:分类后的热力图生成
### 按照structure统计的最大值来绘制热力图
### 没有针对brand进行分类

In [ ]:
# 按照structure统计的最大值来绘制热力图
# 没有针对brand进行分类
# 通过x和y的list来实现
#



import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import datetime

# 定义左上角排除区域的边界常量
EXCLUDE_LEFT_TOP_X_THRESHOLD = 616 * 0.3
EXCLUDE_LEFT_TOP_Y_THRESHOLD = 616 * 0.2

# 创建保存结果的文件夹，包括创建 url_2 子文件夹
def create_output_folder(x, y):
    output_folder = os.path.join(f'D://code//data//{path}//{z}')
    os.makedirs(output_folder, exist_ok=True)

    # 创建 url_2 子文件夹
    url_2_folder = os.path.join(output_folder, 'url_2')
    os.makedirs(url_2_folder, exist_ok=True)
    return output_folder

# 读取Excel文件并处理，返回特定 structure 值的数据
def read_and_process_excel_file_for_structure(x, y):
    file_path = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', f'50%_word_classification_structure-{x}_{y}.xlsx')
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None
    df = pd.read_excel(file_path)
    df_unique = df.drop_duplicates(subset='Image Name')
    structure_counts = df_unique['structure'].value_counts()
    most_common_structure = structure_counts.index[0] if len(structure_counts) > 0 else None
    return df_unique[df_unique['structure'] == most_common_structure] if most_common_structure else None

# 绘制热力图
def plot_heatmap(df, output_folder, most_common_structure, brand='all'):
    if df is None:
        return
    # 获取数量
    if most_common_structure is not None:
        count = df['structure'].value_counts().get(most_common_structure, 0)
    else:
        count = 0
    print(f"Most common structure: {most_common_structure}, Count: {count}")
    # 筛选出特定 structure 值的数据
    filtered_df = df[df['structure'] == most_common_structure]
    # 创建一个新的图形
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    # 绘制矩形框
    ax1.set_xlim(0, 616)
    ax1.set_ylim(616, 0)
    for _, row in filtered_df.iterrows():
        x1, y1, x2, y2 = row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']
        # 排除左上角的文本框
        if x1 <= EXCLUDE_LEFT_TOP_X_THRESHOLD and y1 <= EXCLUDE_LEFT_TOP_Y_THRESHOLD:
            continue
        width = x2 - x1
        height = y2 - y1
        rect = plt.Rectangle((x1, y1), width, height, fill=False, edgecolor='r')
        ax1.add_patch(rect)
    ax1.set_title(f'Bounding Boxes for {most_common_structure}')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')

    # 创建热力图
    heatmap = np.zeros((616, 616))
    for _, row in filtered_df.iterrows():
        x1, y1, x2, y2 = row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']
        # 排除左上角的文本框
        if x1 <= EXCLUDE_LEFT_TOP_X_THRESHOLD and y1 <= EXCLUDE_LEFT_TOP_Y_THRESHOLD:
            continue
        x1, y1 = max(0, min(x1, 615)), max(0, min(y1, 615))
        x2, y2 = max(0, min(x2, 615)), max(0, min(y2, 615))
        heatmap[int(y1):int(y2)+1, int(x1):int(x2)+1] += 1

    # 绘制热力图
    sns.heatmap(heatmap, ax=ax2, cmap='YlOrRd', cbar=True)
    ax2.set_title(f'Heatmap for {most_common_structure}')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')

    # 调整子图之间的间距
    plt.tight_layout()

    # 保存图像
    plt.savefig(os.path.join(output_folder, f'url_2//{x}_{y}_word_structure_{most_common_structure}_brand_{brand}_url_2_hotlayout.png'), dpi=100, bbox_inches='tight')
    plt.close()  # 关闭图形，释放内存

    print(f"已保存热力图 for {most_common_structure}")

# 主函数
if __name__ == "__main__":
    for x in x_list:
        for y in y_list:
            print(f"Processing for x={x}, y={y}...")
            output_folder = create_output_folder(x, y)
            df = read_and_process_excel_file_for_structure(x, y)
            if df is not None:
                structure_counts = df['structure'].value_counts()
                most_common_structure = structure_counts.index[0] if len(structure_counts) > 0 else None
                plot_heatmap(df, output_folder, most_common_structure)

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    print(formatted_time)
    print(formatted_time)
    print(formatted_time)




# step7.3 - 文本:分类后的热力图生成
### 按照structure统计的最大值来绘制热力图
### 按照brand进行分类

In [ ]:
# 按照structure统计的最大值来绘制热力图
# 针对brand进行分类
# 通过x和y的list来实现
## 这里有可能因为brand分类太少的原因, 造成structure没有值, 所以设计了一个跳过的流程



import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import datetime
import numpy as np

# filter_suffix_list = ['filter_1.0_2.0_3.0', 'filter_4.0_5.0_6.0']

def create_output_folder(x, y, filter_suffix):
    output_folder = os.path.join(f'D://code//data//{path}//{z}')
    os.makedirs(output_folder, exist_ok=True)
    return output_folder

def read_excel_file(x, y, filter_suffix):
    file_path = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}',
                             f'50%_word_classification_structure-{x}_{y}_{filter_suffix}.xlsx')

    if not os.path.exists(file_path):
        return None
    df = pd.read_excel(file_path)
    
    # 如果 'structure' 列不存在或为空，则跳过
    if 'structure' not in df.columns or df['structure'].isnull().all():
        print(f"'structure' column is missing or empty in file: {file_path}")
        return None

    # 根据Image Name去重
    df_unique = df.drop_duplicates(subset='Image Name')

    # 统计structure列各个值的数量
    structure_counts = df_unique['structure'].value_counts()
    most_common_structure = structure_counts.index[0] if len(structure_counts) > 0 else None

    if most_common_structure is None:
        print(f"No valid structure found in file: {file_path}")
        return None
    
    print(f"Most common structure: {most_common_structure}, Total count: {structure_counts.get(most_common_structure, 0)}")
    return df_unique[df_unique['structure'] == most_common_structure]

def create_empty_file(file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    pd.DataFrame().to_excel(file_path, index=False)
    print(f"Created empty file: {file_path}")

def plot_heatmaps(df, output_folder, filter_suffix, most_common_structure):
    if df.empty:
        print(f"Skipping empty DataFrame for {filter_suffix}")
        return

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    ax1.set_xlim(0, 616)
    ax1.set_ylim(616, 0)
    for _, row in df.iterrows():
        x1, y1, x2, y2 = row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']
        if x1 <= 616 * 0.3 and y1 <= 616 * 0.2:
            continue
        width, height = x2 - x1, y2 - y1
        rect = plt.Rectangle((x1, y1), width, height, fill=False, edgecolor='r')
        ax1.add_patch(rect)
    ax1.set_title(f'Bounding Boxes ({filter_suffix})')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')

    heatmap = np.zeros((616, 616))
    for _, row in df.iterrows():
        x1, y1, x2, y2 = row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']
        if x1 <= 616 * 0.3 and y1 <= 616 * 0.2:
            continue
        x1, y1 = max(0, min(x1, 615)), max(0, min(y1, 615))
        x2, y2 = max(0, min(x2, 615)), max(0, min(y2, 615))
        heatmap[int(y1):int(y2) + 1, int(x1):int(x2) + 1] += 1

    sns.heatmap(heatmap, ax=ax2, cmap='YlOrRd', cbar=True)
    ax2.set_title(f'Heatmap ({filter_suffix})')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')

    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f'url_2//{x}_{y}_word_structure_{most_common_structure}_brand_{filter_suffix}_url_2_hotlayout.png'), dpi=100, bbox_inches='tight')
    plt.close()

    print(f"已保存热力图 ({filter_suffix})")

if __name__ == "__main__":
    for x in x_list:
        for y in y_list:
            for filter_suffix in filter_suffix_list:
                print(f"Processing for x={x}, y={y}, filter_suffix={filter_suffix}")
                output_folder = create_output_folder(x, y, filter_suffix)
                df = read_excel_file(x, y, filter_suffix)

                if df is None:
                    print(f"File not found or 'structure' column missing in: {os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', f'50%_word_classification_structure-{x}_{y}_{filter_suffix}.xlsx')}")
                    create_empty_file(os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', f'50%_word_classification_structure-{x}_{y}_{filter_suffix}.xlsx'))
                    continue

                # 筛选出最终分层值在filter_values中的记录
                filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
                df = df[df['最终分层'].isin(filter_values)]

                # 获取最常见的结构
                most_common_structure = df['structure'].value_counts().idxmax() if len(df) > 0 else None

                if most_common_structure is not None:
                    plot_heatmaps(df, output_folder, filter_suffix, most_common_structure)

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    print(formatted_time)
    print(formatted_time)
    print(formatted_time)




# step7.4 - 文本:分类后的热力图生成
### 按照structure统计的最大值来绘制热力图
### 根据文本大小绘制热力图
### 没有对brand进行分层


In [ ]:
# 旧版不, 可能会遇到structure为空的报错情况

# # 先针对structure进行统计rank，然后再进行过滤
# # 根据文本大小绘制热力图
# # 没有对brand进行分层
# # 


# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import os
# import datetime

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True

# for item_x in x_list:
#     for item_y in y_list:
#         # 定义文件路径
#         file_path = os.path.join(f'D://code//data//{path}//{z}//{item_x}//grounding_output//{item_y}', f'50%_word_classification_structure-{item_x}_{item_y}_{filter_suffix}.xlsx')
#         if not os.path.exists(file_path):
#             continue

#         # # 如果 'structure' 列不存在或为空，则跳过
#         # if 'structure' not in df.columns or df['structure'].isnull().all():
#         #     print(f"'structure' column is missing or empty in file: {file_path}")
#         #     continue

#         df = pd.read_excel(file_path)

#         if 'structure' in df_unique.columns:
#             structure_counts = df_unique['structure'].value_counts()
#         else:
#             print("The 'structure' column does not exist in the DataFrame. Please check the data source and processing steps.")

#         df_unique = df.drop_duplicates(subset='Image Name')

#         # 统计 structure 列各个值的数量
#         structure_counts = df_unique['structure'].value_counts()
#         if len(structure_counts) > 0:
#             most_common_structure, max_count = structure_counts.index[0], structure_counts.iloc[0]
#             print(f"Most common structure: {most_common_structure}, Total count: {max_count}")
#             new_df = df_unique[df_unique['structure'] == most_common_structure]
#         else:
#             new_df = pd.DataFrame()

#         # 应用过滤
#         new_df = new_df[new_df.apply(filter_by_rectangle, axis=1)]

#         # 替换 Height_Category 列的值
#         new_df['Height_Category'] = new_df['Height_Category'].replace({
#             'Height_>38': 'xl',
#             'Height_18-29': 'm',
#             'Height_29-38': 'l',
#             'Height_<18': 's'
#         })

#         # 删除空值
#         new_df = new_df.dropna(subset=['Height_Category', 'structure'])

#         # 确保必要的列存在
#         required_columns = ['Height_Category', 'txt_x1', 'txt_y1', 'txt_x2', 'txt_y2']
#         if not all(col in new_df.columns for col in required_columns):
#             raise ValueError(f"DataFrame must contain all of these columns: {required_columns}")

#         # 定义输出路径
#         output_dir = f"D://code//data//{path}//{z}"
#         url_4_path = os.path.join(output_dir, 'url_4')
#         os.makedirs(url_4_path, exist_ok=True)

#         # 分组并绘制热力图
#         grouped = new_df.groupby(['Height_Category'])
#         for box_no, group in grouped:
#             heatmap_data = np.zeros((616, 616))

#             # 更新矩形框的热度值
#             for _, row in group.iterrows():
#                 x1, y1, x2, y2 = map(int, [row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']])
#                 x1, x2 = max(0, min(x1, 616)), max(0, min(x2, 616))
#                 y1, y2 = max(0, min(y1, 616)), max(0, min(y2, 616))
#                 heatmap_data[y1:y2, x1:x2] += 1

#             # 创建热力图
#             plt.figure(figsize=(10, 10))
#             sns.heatmap(heatmap_data, cmap='YlOrRd', cbar_kws={'label': 'Frequency'}, square=True)
#             plt.xlabel('X coordinate')
#             plt.ylabel('Y coordinate')
#             plt.gca().set_aspect('equal', adjustable='box')

#             # 保存图像
#             output_path = os.path.join(url_4_path, f"{item_x}_{item_y}_word_structure_{most_common_structure}_brand_all_{box_no}_wordsize_url_4_hotlayout.png".replace("('", "").replace("',)", ""))
#             plt.savefig(output_path, dpi=300, bbox_inches='tight')
#             plt.close()

#             print(f"Heatmap saved to: {output_path}")

# # 显示时间
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(formatted_time)

In [ ]:
# 新版本, 针对structure为空的情况进行处理, 不理会报错, 直接跳过

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import datetime


def filter_by_rectangle(row):
    right, bottom = 616 * 0.3, 616 * 0.2
    if row['txt_x2'] < right and row['txt_y2'] < bottom:
        return False
    return True


for item_x in x_list:
    for item_y in y_list:
        # 定义文件路径
        file_path = os.path.join(f'D://code//data//{path}//{z}//{item_x}//grounding_output//{item_y}', 
                                 f'50%_word_classification_structure-{item_x}_{item_y}_{filter_suffix}.xlsx')
        if not os.path.exists(file_path):
            continue

        df = pd.read_excel(file_path)

        # 判断'structure'列是否存在于df中
        if 'structure' in df.columns:
            df_unique = df.drop_duplicates(subset='Image Name')

            # 统计structure列各个值的数量
            structure_counts = df_unique['structure'].value_counts()
            if len(structure_counts) > 0:
                most_common_structure, max_count = structure_counts.index[0], structure_counts.iloc[0]
                print(f"Most common structure: {most_common_structure}, Total count: {max_count}")
                new_df = df_unique[df_unique['structure'] == most_common_structure]
            else:
                new_df = pd.DataFrame()
        else:
            print("The 'structure' column does not exist in the DataFrame. Please check the data source and processing steps.")
            new_df = pd.DataFrame()

        # 判断'Height_Category'列是否存在于new_df中
        if 'Height_Category' in new_df.columns:
            # 应用过滤
            new_df = new_df[new_df.apply(filter_by_rectangle, axis=1)]

            # 替换Height_Category列的值
            new_df['Height_Category'] = new_df['Height_Category'].replace({
                'Height_>38': 'xl',
                'Height_18-29': 'm',
                'Height_29-38': 'l',
                'Height_<18': 's'
            })

            # 删除空值
            new_df = new_df.dropna(subset=['Height_Category', 'structure'])

            # 确保必要的列存在
            required_columns = ['Height_Category', 'txt_x1', 'txt_y1', 'txt_x2', 'txt_y2']
            if not all(col in new_df.columns for col in required_columns):
                raise ValueError(f"DataFrame must contain all of these columns: {required_columns}")

            # 定义输出路径
            output_dir = f"D://code//data//{path}//{z}"
            url_4_path = os.path.join(output_dir, 'url_4')
            os.makedirs(url_4_path, exist_ok=True)

            # 分组并绘制热力图
            grouped = new_df.groupby(['Height_Category'])
            for box_no, group in grouped:
                heatmap_data = np.zeros((616, 616))

                # 更新矩形框的热度值
                for _, row in group.iterrows():
                    x1, y1, x2, y2 = map(int, [row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']])
                    x1, x2 = max(0, min(x1, 616)), max(0, min(x2, 616))
                    y1, y2 = max(0, min(y1, 616)), max(0, min(y2, 616))
                    heatmap_data[y1:y2, x1:x2] += 1

                # 创建热力图
                plt.figure(figsize=(10, 10))
                sns.heatmap(heatmap_data, cmap='YlOrRd', cbar_kws={'label': 'Frequency'}, square=True)
                plt.xlabel('X coordinate')
                plt.ylabel('Y coordinate')
                plt.gca().set_aspect('equal', adjustable='box')

                # 保存图像
                output_path = os.path.join(url_4_path, f"{item_x}_{item_y}_word_structure_{most_common_structure}_brand_all_{box_no}_wordsize_url_4_hotlayout.png".replace("('", "").replace("',)", ""))
                plt.savefig(output_path, dpi=300, bbox_inches='tight')
                plt.close()

                print(f"Heatmap saved to: {output_path}")
        else:
            print("The 'Height_Category' column does not exist in the DataFrame. Please check the data source and processing steps.")

# 显示时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)




# step7.5 - 文本:分类后的热力图生成
### 按照structure统计的最大值来绘制热力图
### 根据文本大小绘制热力图
### 根据brand进行分层统计

In [ ]:
# 先针对structure进行统计rank，然后再进行过滤
# 根据文本大小绘制热力图
# 根据brand进行分层统计
# 



import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import datetime

# filter_suffix_list = ['filter_1.0_2.0', 'filter_3.0_4.0', 'filter_5.0_6.0']

def filter_by_rectangle(row):
    right, bottom = 616 * 0.3, 616 * 0.2
    if row['txt_x2'] < right and row['txt_y2'] < bottom:
        return False
    return True

def create_empty_file(file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    pd.DataFrame().to_excel(file_path, index=False)
    print(f"Created empty file: {file_path}")

def process_data(item_x, item_y, filter_suffix):
    file_path = os.path.join(f'D://code//data//{path}//{z}//{item_x}//grounding_output//{item_y}', f'50%_word_classification_structure-{item_x}_{item_y}_{filter_suffix}.xlsx')
    if not os.path.exists(file_path):
        return pd.DataFrame(), None  # 返回空 DataFrame 和 None

    df = pd.read_excel(file_path)

    df_unique = df.drop_duplicates(subset='Image Name')
    if 'structure' in df_unique.columns:
        structure_counts = df_unique['structure'].value_counts()
    else:
        print("The 'structure' column does not exist in the DataFrame. Please check the data source and processing steps.")
        structure_counts = pd.Series()  # 设置为一个空的Series对象作为默认值
    
    if len(structure_counts) > 0:
        most_common_structure = structure_counts.index[0]
        print(f"Most common structure: {most_common_structure}, Total count: {structure_counts.iloc[0]}")
        new_df = df_unique[df_unique['structure'] == most_common_structure]
    else:
        return pd.DataFrame(), None  # 返回空 DataFrame 和 None

    required_columns = ['Height_Category', 'txt_x1', 'txt_y1', 'txt_x2', 'txt_y2', '最终分层']
    if not all(col in new_df.columns for col in required_columns):
        print(f"Missing required columns in {file_path}")
        return pd.DataFrame(), None

    new_df = new_df[new_df.apply(filter_by_rectangle, axis=1)]
    new_df['Height_Category'] = new_df['Height_Category'].replace({
        'Height_>38': 'xl',
        'Height_18-29': 'm',
        'Height_29-38': 'l',
        'Height_<18': 's'
    })
    new_df = new_df.dropna(subset=['Height_Category'])
    new_df = new_df.dropna(subset=['structure'])

    filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
    new_df = new_df[new_df['最终分层'].isin(filter_values)]

    return new_df, most_common_structure

def generate_heatmap(group, output_path):
    if group.empty:
        print(f"Skipping empty group for {output_path}")
        return

    heatmap_data = np.zeros((616, 616))
    for _, row in group.iterrows():
        x1, y1, x2, y2 = map(int, [row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']])
        x1, x2 = max(0, min(x1, 616)), max(0, min(x2, 616))
        y1, y2 = max(0, min(y1, 616)), max(0, min(y2, 616))
        heatmap_data[y1:y2, x1:x2] += 1

    plt.figure(figsize=(10, 10))
    sns.heatmap(heatmap_data, cmap='YlOrRd', cbar_kws={'label': 'Frequency'}, square=True)
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Heatmap saved to: {output_path}")

def main():
    for item_x in x_list:
        for item_y in y_list:
            for filter_suffix in filter_suffix_list:
                print(f"Processing: {item_x} - {item_y} - {filter_suffix}")
                df, most_common_structure = process_data(item_x, item_y, filter_suffix)

                if df.empty or most_common_structure is None:
                    print(f"Skipping empty DataFrame for {item_x} - {item_y} - {filter_suffix}")
                    continue

                output_dir = f"D://code//data//{path}//{z}//{item_x}//grounding_output//{item_y}//url_4"
                os.makedirs(output_dir, exist_ok=True)

                grouped = df.groupby(['Height_Category'])
                for box_no, group in grouped:
                    output_path = os.path.join(output_dir, f"{item_x}_{item_y}_word_structure_{most_common_structure}_brand_{filter_suffix}_{box_no}_wordsize_url_4_hotlayout.png".replace("('", "").replace("',)", ""))
                    generate_heatmap(group, output_path)

    print("All heatmaps have been generated.")

if __name__ == "__main__":
    main()
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    print(formatted_time)



# step8.1 - 利用chat进行文本内容统计
### 这里是用旧版prompt,通过list读取
### 全量的structure
### 全量的brand

In [ ]:
# import pandas as pd
# import os
# from tqdm import tqdm
# from openai import OpenAI



# # 设置API密钥和基础URL
# os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
# os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
# )

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True

# # 定义一个函数，使用GPT-4模型对文本进行自动摘要
# def summarize_with_gpt4(text):
#     response = client.chat.completions.create(
#         model="gpt-4o",  # 使用正确的模型名称，如果需要请更改
#         messages=[
#             {"role": "system", "content": f"你是一个非常优秀的电商分析师,现在有这样一个数据集data:\n\n{text}"},
#             {"role": "user", "content": """
#                 # Role 
#                     角色: 电商数据分析师。
#                 # Profile 
#                     简介: 我是一名电商数据分析师，我的工作是通过收集和整理各种信息，找出影响商品销量的因素，并形成报告。
#                 ## Background 
#                     背景: 现在有一些关于同一类商品的描述信息，我需要从中进行归纳总结，找出这些描述信息都是从哪些维度切入的， 并且在这个维度上统计出下钻的一个细分维度。
#                 ## Goals 
#                     目标: 基于我给到的商品描述信息数据集，归纳总结出描述的方向维度，需要特别关注与细化商品本身的特性，并统计这些维度出现的频率。
#                 ## Constrains 
#                     约束条件: 1、时刻保持自己是电商数据分析师的角色, 2、可以进行适当的联想和猜测, 3、举例的时候禁止出现"", 4、统计频率的时候请仔细仔细再仔细
#                 ## Tone 
#                     语气风格: 正式的，客观的，科学的。
#                 ## Skills 
#                     技能: 1、你有出色的文本理解能力,能够理解输入数据的含义 2、你有出色的归纳总结能力,能够归纳总结出数据的描述维度 3、你也有出色的数据统计能力,能够精确的统计出各个维度出现的频次。
#                 ## OutputFormat 
#                     输出格式:以文字方式输出，按照维度，细分维度，细分维度下具体内容举例，细分维度出现频次呈现"""}
#         ],
#     )
#     return response.choices[0].message.content.strip()

# # 遍历 x 和 y 的所有组合
# for x in x_list:
#     for y in y_list:
#         print(f"Processing: {x} - {y}")
        
#         # 读取Excel文件
#         file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification-{x}_{y}.xlsx'
#         if not os.path.exists(file_path):
#             print(f"File not found: {file_path}")
#             continue
        
#         df = pd.read_excel(file_path)
#         # df = df.dropna(subset=['structure'])

#         # 确保 'text' 列中的所有值都是字符串
#         df['text'] = df['text'].astype(str)
        
#         df = df[df.apply(filter_by_rectangle, axis=1)]
#         df_grouped = df.groupby(['Height_Category'])
        
#         # 遍历每个分组，合并文本并进行总结
#         summaries = []
        
#         # 遍历每个分组
#         for (height_category), group in tqdm(df_grouped):
#             # 合并该组的所有文本
#             all_text = " ".join(group['text'].dropna())
#             # print(f"Structure: {structure}")
#             # print(f"Height Category: {height_category}")
#             # print(f"Text: {all_text[:100]}...")  # 只打印前100个字符
            
#             # 使用 GPT-4 进行总结
#             try:
#                 summary = summarize_with_gpt4(all_text)
#                 # print(f"Summary: {summary[:100]}...")  # 只打印前100个字符
#             except Exception as e:
#                 # print(f"Error in summarization: {str(e)}")
#                 summary = "Error in summarization"
            
#             # 将结果添加到列表中
#             summaries.append({
#                 # 'structure': structure,
#                 'Height_Category': height_category,
#                 'text': all_text,
#                 'summary': summary
#             })
        
#         # 创建一个新的DataFrame来存储结果
#         result_df = pd.DataFrame(summaries)
        
#         # 保存结果到Excel文件
#         output_file = f"D://code//data//Lv2期结论//{z}//{x}//url_4//{x}_{y}_word_structure_all_brand_all_wordsummary.xlsx"
#         result_df.to_excel(output_file, index=False)
#         print(f"Results saved to: {output_file}")

# print("All processing completed.")


# import datetime
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(formatted_time)
# print(formatted_time)
# print(formatted_time)




In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
from openai import OpenAI

# 设置API密钥和基础URL
os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_API_BASE"],
)

def filter_by_rectangle(row):
    right, bottom = 616 * 0.3, 616 * 0.2
    return row['txt_x2'] >= right or row['txt_y2'] >= bottom

# 使用GPT-4模型对文本进行自动摘要
def summarize_with_gpt4(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f"你是一个非常优秀的电商分析师,现在有这样一个数据集data:\n\n{text}"},
            {"role": "user", "content": """
                # Role 
                    角色: 电商数据分析师。
                # Profile 
                    简介: 我是一名电商数据分析师，我的工作是通过收集和整理各种信息，找出影响商品销量的因素，并形成报告。
                ## Background 
                    背景: 现在有一些关于同一类商品的描述信息，我需要从中进行归纳总结，找出这些描述信息都是从哪些维度切入的， 并且在这个维度上统计出下钻的一个细分维度。
                ## Goals 
                    目标: 基于我给到的商品描述信息数据集，归纳总结出描述的方向维度，需要特别关注与细化商品本身的特性，并统计这些维度出现的频率。
                ## Constrains 
                    约束条件: 1、时刻保持自己是电商数据分析师的角色, 2、可以进行适当的联想和猜测, 3、举例的时候禁止出现"", 4、统计频率的时候请仔细仔细再仔细
                ## Tone 
                    语气风格: 正式的，客观的，科学的。
                ## Skills 
                    技能: 1、你有出色的文本理解能力,能够理解输入数据的含义 2、你有出色的归纳总结能力,能够归纳总结出数据的描述维度 3、你也有出色的数据统计能力,能够精确的统计出各个维度出现的频次。
                ## OutputFormat 
                    输出格式:以文字方式输出，按照维度，细分维度，细分维度下具体内容举例，细分维度出现频次呈现"""}
        ],
    )
    return response.choices[0].message.content.strip()

# 创建一个空的总 DataFrame
all_data = pd.DataFrame()

# 遍历 x 和 y 的所有组合
for x in x_list:
    for y in y_list:
        print(f"Processing: {x} - {y}")
        
        # 读取Excel文件
        file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification-{x}_{y}.xlsx'
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue
        
        df = pd.read_excel(file_path)
        df['text'] = df['text'].astype(str)  # 确保 'text' 列为字符串
        df = df[df.apply(filter_by_rectangle, axis=1)]
        df_grouped = df.groupby(['Height_Category'])
        
        # 遍历每个分组，合并文本并进行总结
        summaries = []
        for height_category, group in tqdm(df_grouped):
            all_text = " ".join(group['text'].dropna())
            try:
                summary = summarize_with_gpt4(all_text)
            except Exception as e:
                summary = "Error in summarization"
            
            # 将结果添加到列表中
            summaries.append({
                'x': x,
                'y': y,
                'Height_Category': height_category,
                'text': all_text,
                'summary': summary,
                'style': "old",
                'structure': "all",
                'brand': "all"
            })
        
        # 创建一个 DataFrame 来存储当前组合的结果
        result_df = pd.DataFrame(summaries)
        
        # 重新排序列
        result_df = result_df[['x', 'y', 'Height_Category', 'text', 'summary', 'style', 'structure', 'brand']]
        
        # 将当前的结果 DataFrame 添加到总的 DataFrame 中
        all_data = pd.concat([all_data, result_df], ignore_index=True)

# 保存所有数据到一个汇总的Excel文件中
output_file = f"D://code//data//Lv2期结论//{z}//url_4//word_structure_all_brand_all_summary-old.xlsx"
all_data.to_excel(output_file, index=False)
print(f"All data saved to: {output_file}")

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")




# step8.2 - 利用chat进行文本内容统计
### 这里是用旧版prompt,通过list读取
### 先针对structure进行统计rank，然后再进行过滤
### 没有对brand进行分层

In [ ]:


# '''
# 这里是通过读取list形式, 来简化输入的
# '''


# import pandas as pd
# import os
# from tqdm import tqdm
# from openai import OpenAI



# # 设置API密钥和基础URL
# os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
# os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
# )

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True

# # 定义一个函数，使用GPT-4模型对文本进行自动摘要
# def summarize_with_gpt4(text):
#     response = client.chat.completions.create(
#         model="gpt-4o",  # 使用正确的模型名称，如果需要请更改
#         messages=[
#             {"role": "system", "content": f"你是一个非常优秀的电商分析师,现在有这样一个数据集data:\n\n{text}"},
#             {"role": "user", "content": """
#                 # Role 
#                     角色: 电商数据分析师。
#                 # Profile 
#                     简介: 我是一名电商数据分析师，我的工作是通过收集和整理各种信息，找出影响商品销量的因素，并形成报告。
#                 ## Background 
#                     背景: 现在有一些关于同一类商品的描述信息，我需要从中进行归纳总结，找出这些描述信息都是从哪些维度切入的， 并且在这个维度上统计出下钻的一个细分维度。
#                 ## Goals 
#                     目标: 基于我给到的商品描述信息数据集，归纳总结出描述的方向维度，需要特别关注与细化商品本身的特性，并统计这些维度出现的频率。
#                 ## Constrains 
#                     约束条件: 1、时刻保持自己是电商数据分析师的角色, 2、可以进行适当的联想和猜测, 3、举例的时候禁止出现"", 4、统计频率的时候请仔细仔细再仔细
#                 ## Tone 
#                     语气风格: 正式的，客观的，科学的。
#                 ## Skills 
#                     技能: 1、你有出色的文本理解能力,能够理解输入数据的含义 2、你有出色的归纳总结能力,能够归纳总结出数据的描述维度 3、你也有出色的数据统计能力,能够精确的统计出各个维度出现的频次。
#                 ## OutputFormat 
#                     输出格式:以文字方式输出，按照维度，细分维度，细分维度下具体内容举例，细分维度出现频次呈现"""}
#         ],
#     )
#     return response.choices[0].message.content.strip()

# # 遍历 x 和 y 的所有组合
# for x in x_list:
#     for y in y_list:
#         print(f"Processing: {x} - {y}")
        
#         # 读取Excel文件
#         file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}.xlsx'
       
#         df = pd.read_excel(file_path)
#         # 根据 Image Name 去重
#         df_unique = df.drop_duplicates(subset='Image Name')
#         # 统计 structure 列各个值的数量
#         structure_counts = df_unique['structure'].value_counts()
#         if len(structure_counts) > 0:
#             most_common_structure, max_count = structure_counts.index[0], structure_counts.iloc[0]
#             print(f"Most common structure: {most_common_structure}, Total count: {max_count}")
#             new_df = df_unique[df_unique['structure'] == most_common_structure]
#         else:
#             new_df = pd.DataFrame()
        
#         # 确保 'text' 列中的所有值都是字符串
#         new_df.loc[:, 'text'] = new_df['text'].astype(str)
        
#         new_df = new_df[new_df.apply(filter_by_rectangle, axis=1)]
#         # new_df = new_df[['Height_Category']]
#         df_grouped = new_df.groupby(['Height_Category'])
        
#         # 遍历每个分组，合并文本并进行总结
#         summaries = []
        
#         # 遍历每个分组
#         for (height_category), group in tqdm(df_grouped):
#             # 合并该组的所有文本
#             all_text = " ".join(group['text'].dropna())
#             # print(f"Structure: {structure}")
#             # print(f"Height Category: {height_category}")
#             # print(f"Text: {all_text[:100]}...")  # 只打印前100个字符
            
#             # 使用 GPT-4 进行总结
#             try:
#                 summary = summarize_with_gpt4(all_text)
#                 # print(f"Summary: {summary[:100]}...")  # 只打印前100个字符
#             except Exception as e:
#                 # print(f"Error in summarization: {str(e)}")
#                 summary = "Error in summarization"
            
#             # 将结果添加到列表中
#             summaries.append({
#                 # 'structure': structure,
#                 'Height_Category': height_category,
#                 'text': all_text,
#                 'summary': summary
#             })
        
#         # 创建一个新的DataFrame来存储结果
#         result_df = pd.DataFrame(summaries)
        
#         # 保存结果到Excel文件
#         output_file = f"D://code//data//Lv2期结论//{z}//{x}//url_4//{x}_{y}_word_structure_{most_common_structure}_brand_all_wordsummary.xlsx"

#         result_df.to_excel(output_file, index=False)
#         print(f"Results saved to: {output_file}")

# print("All processing completed.")


# import datetime
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(formatted_time)
# print(formatted_time)
# print(formatted_time)




In [ ]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import datetime

# # 定义 x_list 和 y_list
# x_list = ['9736', '9735', '12004']
# y_list = ['txt', 'price']
# path = 'Lv2期结论'
# z = '男士春夏下装_from_0501'

# # 定义过滤层级条件
# filter_layer_cases = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]  # 示例分层条件

# 用于汇总所有的 DataFrame
all_summaries = []

# 遍历 x_list 和 y_list 的每个组合
for x in x_list:
    for y in y_list:
        print(f"Processing: {x} - {y}")
        
        # 读取Excel文件
        file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}.xlsx'
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue
        
        df = pd.read_excel(file_path)
        
        # 去重并统计 structure 列的最常见值
        df_unique = df.drop_duplicates(subset='Image Name')
        structure_counts = df_unique['structure'].value_counts()
        if len(structure_counts) > 0:
            most_common_structure = structure_counts.idxmax()
            print(f"Most common structure: {most_common_structure}")
            new_df = df_unique[df_unique['structure'] == most_common_structure]
        else:
            new_df = pd.DataFrame()

        # 过滤符合条件的数据
        new_df['text'] = new_df['text'].astype(str)
        new_df = new_df[new_df.apply(filter_by_rectangle, axis=1)]
        df_grouped = new_df.groupby(['Height_Category'])

        summaries = []
        for height_category, group in tqdm(df_grouped):
            all_text = " ".join(group['text'].dropna())
            try:
                summary = summarize_with_gpt4(all_text)
            except Exception as e:
                summary = "Error in summarization"

            summaries.append({
                'x': x,
                'y': y,
                'Height_Category': height_category,
                'text': all_text,
                'summary': summary,
                'style': "old",
                'structure': most_common_structure,
                'brand': "all"
            })

        result_df = pd.DataFrame(summaries)
        all_summaries.append(result_df)  # 将每个结果DataFrame添加到汇总列表中

# 汇总所有DataFrame并保存为一个总的Excel文件
final_df = pd.concat(all_summaries, ignore_index=True)
output_file = f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_all_summary-old.xlsx"
final_df.to_excel(output_file, index=False)
print(f"All summaries saved to: {output_file}")

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")




# step8.3 - 利用chat进行文本内容统计
### 这里是用旧版prompt,通过list读取
### 先针对structure进行统计rank，然后再进行过滤
### 对brand实行分层统计

In [ ]:
# 这里把之前的版本折叠了,这个版本是将各个excel单独输出的

# import pandas as pd
# import os
# from tqdm import tqdm
# from openai import OpenAI
# import datetime



# # filter_suffix_list = ['filter_1.0_2.0_3.0','filter_4.0_5.0_6.0']



# # 设置API密钥和基础URL
# os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
# os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
# )

# def create_empty_file(file_path):
#     os.makedirs(os.path.dirname(file_path), exist_ok=True)
#     pd.DataFrame().to_excel(file_path, index=False)
#     print(f"Created empty file: {file_path}")

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True

# # 定义一个函数，使用GPT-4模型对文本进行自动摘要
# def summarize_with_gpt4(text):
#     try:
#         response = client.chat.completions.create(
#             model="gpt-4o",  # 使用正确的模型名称，如果需要请更改
#             messages=[
#                 {"role": "system", "content": f"你是一个非常优秀的电商分析师,现在有这样一个数据集data:\n\n{text}"},
#                 {"role": "user", "content": """
#                     # Role 
#                         角色: 电商数据分析师。
#                     # Profile 
#                         简介: 我是一名电商数据分析师，我的工作是通过收集和整理各种信息，找出影响商品销量的因素，并形成报告。
#                     ## Background 
#                         背景: 现在有一些关于同一类商品的描述信息，我需要从中进行归纳总结，找出这些描述信息都是从哪些维度切入的， 并且在这个维度上统计出下钻的一个细分维度。
#                     ## Goals 
#                         目标: 基于我给到的商品描述信息数据集，归纳总结出描述的方向维度，需要特别关注与细化商品本身的特性，并统计这些维度出现的频率。
#                     ## Constrains 
#                         约束条件: 1、时刻保持自己是电商数据分析师的角色, 2、可以进行适当的联想和猜测, 3、举例的时候禁止出现"", 4、统计频率的时候请仔细仔细再仔细
#                     ## Tone 
#                         语气风格: 正式的，客观的，科学的。
#                     ## Skills 
#                         技能: 1、你有出色的文本理解能力,能够理解输入数据的含义 2、你有出色的归纳总结能力,能够归纳总结出数据的描述维度 3、你也有出色的数据统计能力,能够精确的统计出各个维度出现的频次。
#                     ## OutputFormat 
#                         输出格式:以文字方式输出，按照维度，细分维度，细分维度下具体内容举例，细分维度出现频次呈现"""}
#             ],
#         )
#         return response.choices[0].message.content.strip()
#     except Exception as e:
#         print(f"Error in summarization: {str(e)}")
#         return ""

# # 遍历 x, y 和 filter_suffix 的所有组合
# for x in x_list:
#     for y in y_list:
#         for filter_suffix in filter_suffix_list:
#             print(f"Processing: {x} - {y} - {filter_suffix}")
            
#             # 读取Excel文件
#             file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}_{filter_suffix}.xlsx'
#             df = pd.read_excel(file_path)
#             # 根据 Image Name 去重
#             df_unique = df.drop_duplicates(subset='Image Name')
#             # 统计 structure 列各个值的数量
#             structure_counts = df_unique['structure'].value_counts()
#             if len(structure_counts) > 0:
#                 most_common_structure, max_count = structure_counts.index[0], structure_counts.iloc[0]
#                 print(f"Most common structure: {most_common_structure}, Total count: {max_count}")
#                 new_df = df_unique[df_unique['structure'] == most_common_structure]
#             else:
#                 new_df = pd.DataFrame()

#             # 确保 'text' 列中的所有值都是字符串
#             new_df['text'] = new_df['text'].astype(str)
            
#             # 根据 filter_suffix 筛选数据
#             filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
#             new_df = new_df[new_df['最终分层'].isin(filter_values)]
            
#             new_df = new_df[df.apply(filter_by_rectangle, axis=1)]
#             df_grouped = new_df.groupby(['Height_Category'])
            
#             # 遍历每个分组，合并文本并进行总结
#             summaries = []
            
#             # 遍历每个分组
#             for (height_category), group in tqdm(df_grouped):
#                 # 合并该组的所有文本
#                 all_text = " ".join(group['text'].dropna())
                
#                 # 使用 GPT-4 进行总结
#                 summary = summarize_with_gpt4(all_text)
                
#                 # 将结果添加到列表中
#                 summaries.append({
#                     'Height_Category': height_category,
#                     'text': all_text,
#                     'summary': summary
#                 })
            
#             # 创建一个新的DataFrame来存储结果
#             result_df = pd.DataFrame(summaries)
            
#             # 保存结果到Excel文件，包含 filter_suffix 在文件名中
#             output_file = f"D://code//data//{path}//{z}//url_4//{x}_{y}_word_structure_{most_common_structure}_brand_{filter_suffix}_wordsummary.xlsx"
#             result_df.to_excel(output_file, index=False)
#             print(f"Results saved to: {output_file}")

# print("All processing completed.")

# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(formatted_time)
# print(formatted_time)
# print(formatted_time)




In [ ]:
# 旧版本, 可能会遇到structure为空跳出的情况

# import pandas as pd
# import os
# from tqdm import tqdm
# from openai import OpenAI
# import datetime


# # filter_suffix_list = ['filter_1.0_2.0_3.0','filter_4.0_5.0_6.0']



# # 设置API密钥和基础URL
# os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
# os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
# )

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     return not (row['txt_x2'] < right and row['txt_y2'] < bottom)


# # 用于汇总所有的 DataFrame
# all_summaries = []

# # 遍历 x, y 和 filter_suffix 的所有组合
# for x in x_list:
#     for y in y_list:
#         for filter_suffix in filter_suffix_list:
#             print(f"Processing: {x} - {y} - {filter_suffix}")
            
#             file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}_{filter_suffix}.xlsx'
#             if not os.path.exists(file_path):
#                 print(f"File not found: {file_path}")
#                 continue
            
#             df = pd.read_excel(file_path).drop_duplicates(subset='Image Name')

#             if 'structure' in df.columns:
#                 structure_counts = df['structure'].value_counts()
#             else:
#                 print("The 'structure' column does not exist in the DataFrame. Please check the data source and processing steps.")
            
#             structure_counts = df['structure'].value_counts()
            
#             if len(structure_counts) > 0:
#                 most_common_structure = structure_counts.idxmax()
#                 print(f"Most common structure: {most_common_structure}")
#                 new_df = df[df['structure'] == most_common_structure]
#             else:
#                 new_df = pd.DataFrame()

#             filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
#             new_df = new_df[(new_df['最终分层'].isin(filter_values)) & new_df.apply(filter_by_rectangle, axis=1)]
#             new_df['text'] = new_df['text'].astype(str)

#             df_grouped = new_df.groupby(['Height_Category'])
#             summaries = []

#             for height_category, group in tqdm(df_grouped):
#                 all_text = " ".join(group['text'].dropna())

#                 summary = summarize_with_gpt4(all_text)
#                 summaries.append({
#                     'x': x,
#                     'y': y,
#                     'Height_Category': height_category,
#                     'text': all_text,
#                     'summary': summary,
#                     'style': "old",
#                     'structure': most_common_structure,
#                     'brand': filter_suffix
#                 })

#             result_df = pd.DataFrame(summaries)
#             all_summaries.append(result_df)

#             output_file = f"D://code//data//{path}//{z}//url_4//{x}_{y}_word_structure_{most_common_structure}_brand_{filter_suffix}_wordsummary.xlsx"
#             result_df.to_excel(output_file, index=False)
#             print(f"Results saved to: {output_file}")

# # 汇总所有生成的 DataFrame 并保存为一个总的 Excel 文件
# final_df = pd.concat(all_summaries, ignore_index=True)
# combined_output_file = f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_every_summary-old.xlsx"
# final_df.to_excel(combined_output_file, index=False)
# print(f"All summaries saved to: {combined_output_file}")

# # 打印当前时间
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(f"All processing completed at: {formatted_time}")


In [ ]:
# 新版本, 会跳过structure为空的情况



import pandas as pd
import os
from tqdm import tqdm
from openai import OpenAI
import datetime


# filter_suffix_list = ['filter_1.0_2.0_3.0', 'filter_4.0_5.0_6.0']


# 设置API密钥和基础URL
os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_API_BASE"],
)


def filter_by_rectangle(row):
    right, bottom = 616 * 0.3, 616 * 0.2
    return not (row['txt_x2'] < right and row['txt_y2'] < bottom)


# 用于汇总所有的 DataFrame
all_summaries = []


# 遍历 x, y 和 filter_suffix 的所有组合
for x in x_list:
    for y in y_list:
        for filter_suffix in filter_suffix_list:
            print(f"Processing: {x} - {y} - {filter_suffix}")

            file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}_{filter_suffix}.xlsx'
            if not os.path.exists(file_path):
                print(f"File not found: {file_path}")
                continue

            df = pd.read_excel(file_path).drop_duplicates(subset='Image Name')

            if 'structure' in df.columns:
                structure_counts = df['structure'].value_counts()
                if len(structure_counts) > 0:
                    most_common_structure = structure_counts.idxmax()
                    print(f"Most common structure: {most_common_structure}")
                    new_df = df[df['structure'] == most_common_structure]
                else:
                    new_df = pd.DataFrame()
            else:
                print("The 'structure' column does not exist in the DataFrame. Please check the data source and processing steps.")
                new_df = pd.DataFrame()
                continue  # 遇到 'structure' 列不存在的情况，直接跳过本次循环后续处理，进入下一次循环

            filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
            new_df = new_df[(new_df['最终分层'].isin(filter_values)) & new_df.apply(filter_by_rectangle, axis=1)]
            new_df['text'] = new_df['text'].astype(str)

            df_grouped = new_df.groupby(['Height_Category'])
            summaries = []

            for height_category, group in tqdm(df_grouped):
                all_text = " ".join(group['text'].dropna())

                summary = summarize_with_gpt4(all_text)
                summaries.append({
                    'x': x,
                    'y': y,
                    'Height_Category': height_category,
                    'text': all_text,
                    'summary': summary,
                    'style': "old",
                    'structure': most_common_structure,
                    'brand': filter_suffix
                })

            result_df = pd.DataFrame(summaries)
            all_summaries.append(result_df)

            output_file = f"D://code//data//{path}//{z}//url_4//{x}_{y}_word_structure_{most_common_structure}_brand_{filter_suffix}_wordsummary.xlsx"
            result_df.to_excel(output_file, index=False)
            print(f"Results saved to: {output_file}")

# 汇总所有生成的 DataFrame 并保存为一个总的 Excel 文件
final_df = pd.concat(all_summaries, ignore_index=True)
combined_output_file = f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_every_summary-old.xlsx"
final_df.to_excel(combined_output_file, index=False)
print(f"All summaries saved to: {combined_output_file}")

# 打印当前时间
current_time = datetime.datetime.now()
formulated_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formulated_time}")







# step8.4 - 利用chat进行文本内容统计
### 这里是用新版prompt,通过list读取
### 全量structure
### 全量brand

In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
from openai import OpenAI

# 设置API密钥和基础URL
os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_API_BASE"],
)

def filter_by_rectangle(row):
    right, bottom = 616 * 0.3, 616 * 0.2
    return row['txt_x2'] >= right or row['txt_y2'] >= bottom

# 使用GPT-4模型对文本进行自动摘要
def summarize_with_gpt4_new(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f"你是一个非常优秀的电商分析师,现在有这样一个数据集data:\n\n{text}"},
            {"role": "user", "content": """
                # Role 
                角色: 电商数据分析师。
                # Profile 
                简介: 我是一名电商数据分析师，我的工作是通过收集和整理各种信息，找出影响商品销量的因素，并形成报告。
                ## Background 
                背景: 现在有一些关于同一类商品的描述信息，我需要从中进行归纳总结，并基于一些前置的定义，找出这些描述信息都是从哪些维度切入的。
                ## Goals 
                目标: 基于我给到的商品描述信息数据集和前置的维度定义，归纳总结出描述的方向维度，需要特别关注与细化商品本身的卖点特性，并统计这些维度出现的频率。
                ## Definitions
                定义：
                1. 直接展示价格：直接展示价格信息，到手价，预估到手价，会员价等，通常包含上述前缀，¥+具体的价格数字或者具体的价格数字+元。
                2. 折扣信息：描述商品的折扣，通常包含具体的折扣数字+折。
                3. 直降信息：描述商品相较原价进行了大幅降价，通常包含直降、立减。
                4. 满减信息：描述若购买到一定金额，可以在此基础上进行金额优惠，通常包含满+具体的金额+减+具体的金额
                5. 赠品信息：描述若购买商品则会赠送服务或商品，通常包含赠、送
                6. 限时：描述商品促销的时间，通常包含活动时间段、活动开始时间、活动结束时间
                7. 品牌名称：描述商品的品牌名称
                8. 代言人信息：描述商品的代言人信息
                9. 价保：价格保护，通常包含价保
                10. 店铺背书：描述店铺的信息，通常包含旗舰店、自营
                11. 物流服务：描述商品所包含的物流服务，通常包含物流时效、运费险、物流名称、仓库名称、包邮
                12. 直接展示价格属于价格信息一级维度，折扣信息、直降信息、满减信息、赠品信息、限时属于价促活动一级维度，品牌名称、代言人信息属于品牌信息一级维度，价保、店铺背书、物流服务属于服务保障一级维度
                ## Constrains 
                约束条件: 1、时刻保持自己是电商数据分析师的角色，2、可以进行适当的联想和猜测，3、举例的时候禁止出现""，4、统计频率的时候请仔细仔细再仔细，5、若识别到的内容不在上述定义的维度中，可自行命名并统计，请不要忽视未被定义的维度，特别是关于商品本身的卖点信息描述
                ## Tone 
                语气风格: 正式的，客观的，科学的。
                ## Skills 
                技能: 1、你有出色的文本理解能力，能够理解输入数据的含义 2、你有出色的归纳总结能力，能够归纳总结出数据的描述维度 3、你也有出色的数据统计能力，能够精确的统计出各个维度出现的频次。
                ## OutputFormat 
                输出格式:以文字方式输出，一级维度，一级维度下具体内容和举例和频次，输出顺序按照价格信息、价促活动、品牌信息、服务保障、商品卖点进行输出，商品卖点为未定义维度，请你依照自己的知识库信息进行汇总输出，需要特别注意，是关于商品本身的描述，输出格式为1.价格信息 总频次 直接展示价格 频次 举例 以此类推,注意输出要精简，减少不必要的换行
                    """}
        ],
    )
    return response.choices[0].message.content.strip()

# 创建一个空的总 DataFrame
all_data = pd.DataFrame()

# 遍历 x 和 y 的所有组合
for x in x_list:
    for y in y_list:
        print(f"Processing: {x} - {y}")
        
        # 读取Excel文件
        file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification-{x}_{y}.xlsx'
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue
        
        df = pd.read_excel(file_path)
        df['text'] = df['text'].astype(str)  # 确保 'text' 列为字符串
        df = df[df.apply(filter_by_rectangle, axis=1)]
        df_grouped = df.groupby(['Height_Category'])
        
        # 遍历每个分组，合并文本并进行总结
        summaries = []
        for height_category, group in tqdm(df_grouped):
            all_text = " ".join(group['text'].dropna())
            try:
                summary = summarize_with_gpt4_new(all_text)
            except Exception as e:
                summary = "Error in summarization"
            
            # 将结果添加到列表中
            summaries.append({
                'x': x,
                'y': y,
                'Height_Category': height_category,
                'text': all_text,
                'summary': summary,
                'style': "new",
                'structure': "all",
                'brand': "all"
            })
        
        # 创建一个 DataFrame 来存储当前组合的结果
        result_df = pd.DataFrame(summaries)
        
        # 重新排序列
        result_df = result_df[['x', 'y', 'Height_Category', 'text', 'summary', 'style', 'structure', 'brand']]
        
        # 将当前的结果 DataFrame 添加到总的 DataFrame 中
        all_data = pd.concat([all_data, result_df], ignore_index=True)

# 保存所有数据到一个汇总的Excel文件中
output_file = f"D://code//data//Lv2期结论//{z}//url_4//word_structure_all_brand_all_summary-new.xlsx"
all_data.to_excel(output_file, index=False)
print(f"All data saved to: {output_file}")

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")




# step8.5 - 利用chat进行文本内容统计
### 这里是用新版prompt,通过list读取
### 先针对structure进行统计rank，然后再进行过滤
### 没有对brand进行分层

In [ ]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import datetime

# # 定义 x_list 和 y_list
# x_list = ['9736', '9735', '12004']
# y_list = ['txt', 'price']
# path = 'Lv2期结论'
# z = '男士春夏下装_from_0501'

# # 定义过滤层级条件
# filter_layer_cases = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]  # 示例分层条件

# 用于汇总所有的 DataFrame
all_summaries = []

# 遍历 x_list 和 y_list 的每个组合
for x in x_list:
    for y in y_list:
        print(f"Processing: {x} - {y}")
        
        # 读取Excel文件
        file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}.xlsx'
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue
        
        df = pd.read_excel(file_path)
        
        # 去重并统计 structure 列的最常见值
        df_unique = df.drop_duplicates(subset='Image Name')
        structure_counts = df_unique['structure'].value_counts()
        if len(structure_counts) > 0:
            most_common_structure = structure_counts.idxmax()
            print(f"Most common structure: {most_common_structure}")
            new_df = df_unique[df_unique['structure'] == most_common_structure]
        else:
            new_df = pd.DataFrame()

        # 过滤符合条件的数据
        new_df['text'] = new_df['text'].astype(str)
        new_df = new_df[new_df.apply(filter_by_rectangle, axis=1)]
        df_grouped = new_df.groupby(['Height_Category'])

        summaries = []
        for height_category, group in tqdm(df_grouped):
            all_text = " ".join(group['text'].dropna())
            try:
                summary = summarize_with_gpt4_new(all_text)
            except Exception as e:
                summary = "Error in summarization"

            summaries.append({
                'x': x,
                'y': y,
                'Height_Category': height_category,
                'text': all_text,
                'summary': summary,
                'style': "new",
                'structure': most_common_structure,
                'brand': "all"
            })

        result_df = pd.DataFrame(summaries)
        all_summaries.append(result_df)  # 将每个结果DataFrame添加到汇总列表中

# 汇总所有DataFrame并保存为一个总的Excel文件
final_df = pd.concat(all_summaries, ignore_index=True)
output_file = f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_all_summary-new.xlsx"
final_df.to_excel(output_file, index=False)
print(f"All summaries saved to: {output_file}")

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")




# step8.6 - 利用chat进行文本内容统计
### 这里是用新版prompt,通过list读取
### 先针对structure进行统计rank，然后再进行过滤
### 对brand进行了分层处理

In [ ]:
# 旧版本, 可能会遇到structure为空跳出的情况


# import pandas as pd
# import os
# from tqdm import tqdm
# from openai import OpenAI
# import datetime


# # filter_suffix_list = ['filter_1.0_2.0_3.0','filter_4.0_5.0_6.0']



# # 设置API密钥和基础URL
# os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
# os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
# )

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     return not (row['txt_x2'] < right and row['txt_y2'] < bottom)


# # 用于汇总所有的 DataFrame
# all_summaries = []

# # 遍历 x, y 和 filter_suffix 的所有组合
# for x in x_list:
#     for y in y_list:
#         for filter_suffix in filter_suffix_list:
#             print(f"Processing: {x} - {y} - {filter_suffix}")
            
#             file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}_{filter_suffix}.xlsx'
#             if not os.path.exists(file_path):
#                 print(f"File not found: {file_path}")
#                 continue
            
#             df = pd.read_excel(file_path).drop_duplicates(subset='Image Name')
#             structure_counts = df['structure'].value_counts()
#             if len(structure_counts) > 0:
#                 most_common_structure = structure_counts.idxmax()
#                 print(f"Most common structure: {most_common_structure}")
#                 new_df = df[df['structure'] == most_common_structure]
#             else:
#                 new_df = pd.DataFrame()

#             filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
#             new_df = new_df[(new_df['最终分层'].isin(filter_values)) & new_df.apply(filter_by_rectangle, axis=1)]
#             new_df['text'] = new_df['text'].astype(str)

#             df_grouped = new_df.groupby(['Height_Category'])
#             summaries = []

#             for height_category, group in tqdm(df_grouped):
#                 all_text = " ".join(group['text'].dropna())
#                 summary = summarize_with_gpt4_new(all_text)

#                 summaries.append({
#                     'x': x,
#                     'y': y,
#                     'Height_Category': height_category,
#                     'text': all_text,
#                     'summary': summary,
#                     'style': "old",
#                     'structure': most_common_structure,
#                     'brand': filter_suffix
#                 })

#             result_df = pd.DataFrame(summaries)
#             all_summaries.append(result_df)

#             output_file = f"D://code//data//{path}//{z}//url_4//{x}_{y}_word_structure_{most_common_structure}_brand_{filter_suffix}_wordsummary.xlsx"
#             result_df.to_excel(output_file, index=False)
#             print(f"Results saved to: {output_file}")

# # 汇总所有生成的 DataFrame 并保存为一个总的 Excel 文件
# final_df = pd.concat(all_summaries, ignore_index=True)
# combined_output_file = f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_every_summary-new.xlsx"
# final_df.to_excel(combined_output_file, index=False)
# print(f"All summaries saved to: {combined_output_file}")

# # 打印当前时间
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(f"All processing completed at: {formatted_time}")




In [ ]:
# 新版本, 针对structure为空的情况进行处理, 直接跳过
#


import pandas as pd
import os
from tqdm import tqdm
from openai import OpenAI
import datetime


# filter_suffix_list = ['filter_1.0_2.0_3.0', 'filter_4.0_5.0_6.0']


# 设置API密钥和基础URL
os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_API_BASE"],
)


def filter_by_rectangle(row):
    right, bottom = 616 * 0.3, 616 * 0.2
    return not (row['txt_x2'] < right and row['txt_y2'] < bottom)


# 用于汇总所有的 DataFrame
all_summaries = []


# 遍历 x, y 和 filter_suffix 的所有组合
for x in x_list:
    for y in y_list:
        for filter_suffix in filter_suffix_list:
            print(f"Processing: {x} - {y} - {filter_suffix}")

            file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification_structure-{x}_{y}_{filter_suffix}.xlsx'
            if not os.path.exists(file_path):
                print(f"File not found: {file_path}")
                continue

            df = pd.read_excel(file_path).drop_duplicates(subset='Image Name')

            if 'structure' in df.columns:
                structure_counts = df['structure'].value_counts()
                if len(structure_counts) > 0:
                    most_common_structure = structure_counts.idxmax()
                    print(f"Most common structure: {most_common_structure}")
                    new_df = df[df['structure'] == most_common_structure]
                else:
                    new_df = pd.DataFrame()
            else:
                print("The 'structure' column does not exist in the DataFrame. Please check the data source and processing steps.")
                new_df = pd.DataFrame()
                continue  # 遇到 'structure' 列不存在的情况，直接跳过本次循环后续处理，进入下一次循环

            filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
            new_df = new_df[(new_df['最终分层'].isin(filter_values)) & new_df.apply(filter_by_rectangle, axis=1)]
            new_df['text']: new_df['text'].astype(str)

            df_grouped = new_df.groupby(['Height_Category'])
            summaries = []

            for height_category, group in tqdm(df_grouped):
                all_text = " ".join(group['text'].dropna())
                summary = summarize_with_gpt4_new(all_text)

                summaries.append({
                    'x': x,
                    'y': y,
                    'Height_Category': height_category,
                    'text': all_text,
                    'summary': summary,
                    'style': "old",
                    'structure': most_common_structure,
                    'brand': filter_suffix
                })

            result_df = pd.DataFrame(summaries)
            all_summaries.append(result_df)

            output_file = f"D://code//data//{path}//{z}//url_4//{x}_{y}_word_structure_{most_common_structure}_brand_{filter_suffix}_wordsummary.xlsx"
            result_df.to_excel(output_file, index=False)
            print(f"Results saved to: {output_file}")

# 汇总所有生成的 DataFrame 并保存为一个总的 Excel 文件
final_df = pd.concat(all_summaries, ignore_index=True)
combined_output_file = f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_every_summary-new.xlsx"
final_df.to_excel(combined_output_file, index=False)
print(f"All summaries saved to: {combined_output_file}")

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")
print(f"All processing completed at: {formatted_time}")
print(f"All processing completed at: {formatted_time}")




In [ ]:
import pandas as pd

# 指定要拼接的Excel文件路径列表
excel_file_paths = [
    f"D://code//data//Lv2期结论//{z}//url_4//word_structure_all_brand_all_summary-old.xlsx",
    f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_all_summary-old.xlsx",
    f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_every_summary-old.xlsx",
    f"D://code//data//Lv2期结论//{z}//url_4//word_structure_all_brand_all_summary-new.xlsx",
    f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_all_summary-new.xlsx",
    f"D://code//data//Lv2期结论//{z}//url_4//word_structure_every_brand_every_summary-new.xlsx"
]

# 用于存储读取的每个Excel文件的数据框
dataframes = []

# 逐个读取指定的Excel文件并添加到dataframes列表中
for file_path in excel_file_paths:
    df = pd.read_excel(file_path)
    dataframes.append(df)

# 将所有数据框上下拼接在一起
merged_df = pd.concat(dataframes, axis=0, ignore_index=True)

# 可以根据需要将拼接后的结果保存为新的Excel文件
merged_df.to_excel(f"D://code//data//Lv2期结论//{z}//url_4//word_summary-all.xlsx", index=False)

# 打印当前时间
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"All processing completed at: {formatted_time}")
print(f"All processing completed at: {formatted_time}")
print(f"All processing completed at: {formatted_time}")




In [ ]:
# import os
# import pandas as pd
# import matplotlib.pyplot as plt
# from matplotlib.patches import Rectangle
# import seaborn as sns
# import datetime
# import numpy as np



# # filter_suffix_list = ['filter_1.0_2.0','filter_3.0_4.0','filter_5.0_6.0']



# def create_output_folder(x, y, filter_suffix):
#     output_folder = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', f'50%_word_heatmaps_{filter_suffix}')
#     os.makedirs(output_folder, exist_ok=True)
#     return output_folder

# def read_excel_file(x, y, filter_suffix):
#     file_path = os.path.join(f'D://code//data//{path}//{z}//{x}//grounding_output//{y}', f'50%_word_classification-{x}_{y}_{filter_suffix}.xlsx')
#     return file_path

# def create_empty_file(file_path):
#     os.makedirs(os.path.dirname(file_path), exist_ok=True)
#     pd.DataFrame().to_excel(file_path, index=False)
#     print(f"Created empty file: {file_path}")

# def plot_heatmaps(df, output_folder, filter_suffix):
#     if df.empty:
#         print(f"Skipping empty DataFrame for {filter_suffix}")
#         return

#     grouped = df.groupby('Classification')
#     for name, group in grouped:
#         fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        
#         ax1.set_xlim(0, 616)
#         ax1.set_ylim(616, 0)
#         for _, row in group.iterrows():
#             x1, y1, x2, y2 = row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']
#             if x1 <= 616 * 0.3 and y1 <= 616 * 0.2:
#                 continue
#             width, height = x2 - x1, y2 - y1
#             rect = plt.Rectangle((x1, y1), width, height, fill=False, edgecolor='r')
#             ax1.add_patch(rect)
#         ax1.set_title(f'Bounding Boxes for {name} ({filter_suffix})')
#         ax1.set_xlabel('X')
#         ax1.set_ylabel('Y')

#         heatmap = np.zeros((616, 616))
#         for _, row in group.iterrows():
#             x1, y1, x2, y2 = row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']
#             if x1 <= 616 * 0.3 and y1 <= 616 * 0.2:
#                 continue
#             x1, y1 = max(0, min(x1, 615)), max(0, min(y1, 615))
#             x2, y2 = max(0, min(x2, 615)), max(0, min(y2, 615))
#             heatmap[int(y1):int(y2)+1, int(x1):int(x2)+1] += 1

#         sns.heatmap(heatmap, ax=ax2, cmap='YlOrRd', cbar=True)
#         ax2.set_title(f'Heatmap for {name} ({filter_suffix})')
#         ax2.set_xlabel('X')
#         ax2.set_ylabel('Y')

#         plt.tight_layout()
#         plt.savefig(os.path.join(output_folder, f'{name}_{filter_suffix}_heatmap.png'), dpi=100, bbox_inches='tight')
#         plt.close()

#         print(f"已保存 {name} 的热力图 ({filter_suffix})")

#     print(f"所有热力图已保存在 {output_folder} 文件夹中")

# if __name__ == "__main__":
#     for x in x_list:
#         for y in y_list:
#             for filter_suffix in filter_suffix_list:
#                 print(f"Processing for x={x}, y={y}, filter_suffix={filter_suffix}...")
#                 output_folder = create_output_folder(x, y, filter_suffix)
#                 file_path = read_excel_file(x, y, filter_suffix)
                
#                 try:
#                     df = pd.read_excel(file_path)
#                 except FileNotFoundError:
#                     print(f"File not found: {file_path}")
#                     create_empty_file(file_path)
#                     df = pd.DataFrame()

#                 if not df.empty:
#                     filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
#                     df = df[df['最终分层'].isin(filter_values)]
                
#                 plot_heatmaps(df, output_folder, filter_suffix)

#     current_time = datetime.datetime.now()
#     formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
#     print(formatted_time)
#     print(formatted_time)
#     print(formatted_time)




# 文本大小的总结

In [ ]:
# # 新流程, 可以通过list方式, 来合并读取


# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import os



# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True


# for item_x in x_list:
#     for item_y in y_list:
#         # 读取Excel文件
#         file_path = f'D://code//data//{path}//{z}//{item_x}//grounding_output//{item_y}//50%_word_classification-{item_x}_{item_y}.xlsx'
#         if not os.path.exists(file_path):
#             print(f"No data found for x={item_x}, y={item_y}. Skipping...")
#             continue
#         df = pd.read_excel(file_path)
        
#         # 应用 filter_by_rectangle 函数来过滤数据
#         df = df[df.apply(filter_by_rectangle, axis=1)]

#         # 替换 Height_Category 列的值
#         df['Height_Category'] = df['Height_Category'].replace({
#             'Height_>38': 'Height大于38',
#             'Height_18-29': 'Height18到29',
#             'Height_29-38': 'Height29到38',
#             'Height_<18': 'Height小于18'
#         })

#         # 删除 structure 为空值的行
#         # df = df.dropna(subset=['structure'])
#         df = df.dropna(subset=['Height_Category'])

#         # 确保必要的列存在
#         required_columns = ['Height_Category', 'txt_x1', 'txt_y1', 'txt_x2', 'txt_y2']
#         if not all(col in df.columns for col in required_columns):
#             raise ValueError(f"DataFrame must contain all of these columns: {required_columns}")

#         # 创建输出目录
#         output_dir = f"D://code//data//{path}//{z}//{item_x}//grounding_output//{item_y}//50%_word_wordsize_heatmaps"
#         os.makedirs(output_dir, exist_ok=True)

#         # 按 structure 和 box_no 分组
#         grouped = df.groupby(['Height_Category'])

#         # 遍历每个分组
#         for (box_no), group in grouped:
#             # 创建一个空的 2D numpy 数组来存储热力图数据，大小为 616x616
#             heatmap_data = np.zeros((616, 616))

#             # 对每个矩形框增加热度值
#             for _, row in group.iterrows():
#                 x1, y1, x2, y2 = map(int, [row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']])
#                 # 确保坐标不超出边界
#                 x1, x2 = max(0, min(x1, 616)), max(0, min(x2, 616))
#                 y1, y2 = max(0, min(y1, 616)), max(0, min(y2, 616))
#                 heatmap_data[y1:y2, x1:x2] += 1

#             # 创建图形，设置大小为正方形
#             plt.figure(figsize=(10, 10))

#             # 使用 seaborn 绘制热力图
#             sns.heatmap(heatmap_data, cmap='YlOrRd', cbar_kws={'label': 'Frequency'}, square=True)

#             # 设置标题和轴标签
#             # plt.title(f'Bounding Box Heatmap - Structure: {structure}, word size: {Height_Category}')
#             plt.xlabel('X coordinate')
#             plt.ylabel('Y coordinate')

#             # 调整图形以保持正方形比例
#             plt.gca().set_aspect('equal', adjustable='box')

#             # 保存图形
#             output_path = os.path.join(output_dir, f"heatmap_wordsize_{box_no}.png")
#             plt.savefig(output_path, dpi=300, bbox_inches='tight')
#             plt.close()

#             print(f"Heatmap saved to: {output_path}")

# print("All heatmaps have been generated.")

# import datetime
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(formatted_time)
# print(formatted_time)
# print(formatted_time)




In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import os
# import datetime



# # filter_suffix_list = ['filter_1.0_2.0','filter_3.0_4.0','filter_5.0_6.0']



# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True

# def create_empty_file(file_path):
#     os.makedirs(os.path.dirname(file_path), exist_ok=True)
#     pd.DataFrame().to_excel(file_path, index=False)
#     print(f"Created empty file: {file_path}")

# def process_data(item_x, item_y, filter_suffix):
#     file_path = f'D://code//data//{path}//{z}//{item_x}//grounding_output//{item_y}//50%_word_classification-{item_x}_{item_y}_{filter_suffix}.xlsx'
    
#     try:
#         df = pd.read_excel(file_path)
#     except FileNotFoundError:
#         print(f"File not found: {file_path}")
#         create_empty_file(file_path)
#         return pd.DataFrame()
#     except Exception as e:
#         print(f"Error reading file {file_path}: {str(e)}")
#         return pd.DataFrame()

#     if df.empty:
#         print(f"Empty DataFrame for {file_path}")
#         return df

#     required_columns = ['Height_Category', 'txt_x1', 'txt_y1', 'txt_x2', 'txt_y2', '最终分层']
#     if not all(col in df.columns for col in required_columns):
#         print(f"Missing required columns in {file_path}")
#         return pd.DataFrame()

#     df = df[df.apply(filter_by_rectangle, axis=1)]
#     df['Height_Category'] = df['Height_Category'].replace({
#         'Height_>38': 'Height大于38',
#         'Height_18-29': 'Height18到29',
#         'Height_29-38': 'Height29到38',
#         'Height_<18': 'Height小于18'
#     })
#     df = df.dropna(subset=['Height_Category'])

#     filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
#     df = df[df['最终分层'].isin(filter_values)]

#     return df

# def generate_heatmap(group, output_path):
#     if group.empty:
#         print(f"Skipping empty group for {output_path}")
#         return

#     heatmap_data = np.zeros((616, 616))
#     for _, row in group.iterrows():
#         x1, y1, x2, y2 = map(int, [row['txt_x1'], row['txt_y1'], row['txt_x2'], row['txt_y2']])
#         x1, x2 = max(0, min(x1, 616)), max(0, min(x2, 616))
#         y1, y2 = max(0, min(y1, 616)), max(0, min(y2, 616))
#         heatmap_data[y1:y2, x1:x2] += 1

#     plt.figure(figsize=(10, 10))
#     sns.heatmap(heatmap_data, cmap='YlOrRd', cbar_kws={'label': 'Frequency'}, square=True)
#     plt.xlabel('X coordinate')
#     plt.ylabel('Y coordinate')
#     plt.gca().set_aspect('equal', adjustable='box')
#     plt.savefig(output_path, dpi=300, bbox_inches='tight')
#     plt.close()
#     print(f"Heatmap saved to: {output_path}")

# def main():
#     for item_x in x_list:
#         for item_y in y_list:
#             for filter_suffix in filter_suffix_list:
#                 print(f"Processing: {item_x} - {item_y} - {filter_suffix}")
#                 df = process_data(item_x, item_y, filter_suffix)
                
#                 if df.empty:
#                     print(f"Skipping empty DataFrame for {item_x} - {item_y} - {filter_suffix}")
#                     continue

#                 output_dir = f"D://code//data//{path}//{z}//{item_x}//grounding_output//{item_y}//50%_word_wordsize_heatmaps_{filter_suffix}"
#                 os.makedirs(output_dir, exist_ok=True)

#                 grouped = df.groupby(['Height_Category'])
#                 for box_no, group in grouped:
#                     output_path = os.path.join(output_dir, f"heatmap_wordsize_{box_no}_{filter_suffix}.png")
#                     generate_heatmap(group, output_path)

#     print("All heatmaps have been generated.")

# if __name__ == "__main__":
#     main()
#     current_time = datetime.datetime.now()
#     formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
#     print(formatted_time)
#     print(formatted_time)
#     print(formatted_time)

    

In [ ]:
# # 这里是用旧版prompt,通过list读取,针对整体

# '''
# 这里是通过读取list形式, 来简化输入的
# '''


# import pandas as pd
# import os
# from tqdm import tqdm
# from openai import OpenAI



# # 设置API密钥和基础URL
# os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
# os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
# )

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True

# # 定义一个函数，使用GPT-4模型对文本进行自动摘要
# def summarize_with_gpt4(text):
#     response = client.chat.completions.create(
#         model="gpt-4o",  # 使用正确的模型名称，如果需要请更改
#         messages=[
#             {"role": "system", "content": f"你是一个非常优秀的电商分析师,现在有这样一个数据集data:\n\n{text}"},
#             {"role": "user", "content": """
#                 # Role 
#                     角色: 电商数据分析师。
#                 # Profile 
#                     简介: 我是一名电商数据分析师，我的工作是通过收集和整理各种信息，找出影响商品销量的因素，并形成报告。
#                 ## Background 
#                     背景: 现在有一些关于同一类商品的描述信息，我需要从中进行归纳总结，找出这些描述信息都是从哪些维度切入的， 并且在这个维度上统计出下钻的一个细分维度。
#                 ## Goals 
#                     目标: 基于我给到的商品描述信息数据集，归纳总结出描述的方向维度，需要特别关注与细化商品本身的特性，并统计这些维度出现的频率。
#                 ## Constrains 
#                     约束条件: 1、时刻保持自己是电商数据分析师的角色, 2、可以进行适当的联想和猜测, 3、举例的时候禁止出现"", 4、统计频率的时候请仔细仔细再仔细
#                 ## Tone 
#                     语气风格: 正式的，客观的，科学的。
#                 ## Skills 
#                     技能: 1、你有出色的文本理解能力,能够理解输入数据的含义 2、你有出色的归纳总结能力,能够归纳总结出数据的描述维度 3、你也有出色的数据统计能力,能够精确的统计出各个维度出现的频次。
#                 ## OutputFormat 
#                     输出格式:以文字方式输出，按照维度，细分维度，细分维度下具体内容举例，细分维度出现频次呈现"""}
#         ],
#     )
#     return response.choices[0].message.content.strip()

# # 遍历 x 和 y 的所有组合
# for x in x_list:
#     for y in y_list:
#         print(f"Processing: {x} - {y}")
        
#         # 读取Excel文件
#         file_path = f'D://code//data//Lv2期结论//{z}//{x}//{path}//{y}//50%_word_classification-{x}_{y}.xlsx'
#         if not os.path.exists(file_path):
#             print(f"File not found: {file_path}")
#             continue
        
#         df = pd.read_excel(file_path)
#         # df = df.dropna(subset=['structure'])

#         # 确保 'text' 列中的所有值都是字符串
#         df['text'] = df['text'].astype(str)
        
#         df = df[df.apply(filter_by_rectangle, axis=1)]
#         df_grouped = df.groupby(['Height_Category'])
        
#         # 遍历每个分组，合并文本并进行总结
#         summaries = []
        
#         # 遍历每个分组
#         for (height_category), group in tqdm(df_grouped):
#             # 合并该组的所有文本
#             all_text = " ".join(group['text'].dropna())
#             # print(f"Structure: {structure}")
#             # print(f"Height Category: {height_category}")
#             # print(f"Text: {all_text[:100]}...")  # 只打印前100个字符
            
#             # 使用 GPT-4 进行总结
#             try:
#                 summary = summarize_with_gpt4(all_text)
#                 # print(f"Summary: {summary[:100]}...")  # 只打印前100个字符
#             except Exception as e:
#                 # print(f"Error in summarization: {str(e)}")
#                 summary = "Error in summarization"
            
#             # 将结果添加到列表中
#             summaries.append({
#                 # 'structure': structure,
#                 'Height_Category': height_category,
#                 'text': all_text,
#                 'summary': summary
#             })
        
#         # 创建一个新的DataFrame来存储结果
#         result_df = pd.DataFrame(summaries)
        
#         # 保存结果到Excel文件
#         output_file = f"D://code//data//Lv2期结论//{z}//{x}//grounding_output//{y}//50%_{x}_{y}_文本分类总结.xlsx"
#         result_df.to_excel(output_file, index=False)
#         print(f"Results saved to: {output_file}")

# print("All processing completed.")


# import datetime
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(formatted_time)
# print(formatted_time)
# print(formatted_time)



In [ ]:
# # 这里是用旧版prompt,通过list读取,针对的是品牌维度的分类


# '''
# 这里是通过读取list形式, 来简化输入的
# 添加了针对品牌维度的分类
# '''

# import pandas as pd
# import os
# from tqdm import tqdm
# from openai import OpenAI
# import datetime



# # filter_suffix_list = ['filter_1.0_2.0','filter_3.0_4.0','filter_5.0_6.0']




# # 设置API密钥和基础URL
# os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
# os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
# )

# def create_empty_file(file_path):
#     os.makedirs(os.path.dirname(file_path), exist_ok=True)
#     pd.DataFrame().to_excel(file_path, index=False)
#     print(f"Created empty file: {file_path}")

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True

# # 定义一个函数，使用GPT-4模型对文本进行自动摘要
# def summarize_with_gpt4(text):
#     try:
#         response = client.chat.completions.create(
#             model="gpt-4o",  # 使用正确的模型名称，如果需要请更改
#             messages=[
#                 {"role": "system", "content": f"你是一个非常优秀的电商分析师,现在有这样一个数据集data:\n\n{text}"},
#                 {"role": "user", "content": """
#                     # Role 
#                         角色: 电商数据分析师。
#                     # Profile 
#                         简介: 我是一名电商数据分析师，我的工作是通过收集和整理各种信息，找出影响商品销量的因素，并形成报告。
#                     ## Background 
#                         背景: 现在有一些关于同一类商品的描述信息，我需要从中进行归纳总结，找出这些描述信息都是从哪些维度切入的， 并且在这个维度上统计出下钻的一个细分维度。
#                     ## Goals 
#                         目标: 基于我给到的商品描述信息数据集，归纳总结出描述的方向维度，需要特别关注与细化商品本身的特性，并统计这些维度出现的频率。
#                     ## Constrains 
#                         约束条件: 1、时刻保持自己是电商数据分析师的角色, 2、可以进行适当的联想和猜测, 3、举例的时候禁止出现"", 4、统计频率的时候请仔细仔细再仔细
#                     ## Tone 
#                         语气风格: 正式的，客观的，科学的。
#                     ## Skills 
#                         技能: 1、你有出色的文本理解能力,能够理解输入数据的含义 2、你有出色的归纳总结能力,能够归纳总结出数据的描述维度 3、你也有出色的数据统计能力,能够精确的统计出各个维度出现的频次。
#                     ## OutputFormat 
#                         输出格式:以文字方式输出，按照维度，细分维度，细分维度下具体内容举例，细分维度出现频次呈现"""}
#             ],
#         )
#         return response.choices[0].message.content.strip()
#     except Exception as e:
#         print(f"Error in summarization: {str(e)}")
#         return ""

# # 遍历 x, y 和 filter_suffix 的所有组合
# for x in x_list:
#     for y in y_list:
#         for filter_suffix in filter_suffix_list:
#             print(f"Processing: {x} - {y} - {filter_suffix}")
            
#             # 读取Excel文件
#             file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification-{x}_{y}_{filter_suffix}.xlsx'
#             if not os.path.exists(file_path):
#                 print(f"File not found: {file_path}")
#                 create_empty_file(file_path)
#                 continue
            
#             try:
#                 df = pd.read_excel(file_path)
#             except Exception as e:
#                 print(f"Error reading file {file_path}: {str(e)}")
#                 create_empty_file(file_path)
#                 continue

#             if df.empty:
#                 print(f"Empty DataFrame for {file_path}")
#                 continue

#             # 确保 'text' 列中的所有值都是字符串
#             df['text'] = df['text'].astype(str)
            
#             # 根据 filter_suffix 筛选数据
#             filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
#             df = df[df['最终分层'].isin(filter_values)]
            
#             df = df[df.apply(filter_by_rectangle, axis=1)]
#             df_grouped = df.groupby(['Height_Category'])
            
#             # 遍历每个分组，合并文本并进行总结
#             summaries = []
            
#             # 遍历每个分组
#             for (height_category), group in tqdm(df_grouped):
#                 # 合并该组的所有文本
#                 all_text = " ".join(group['text'].dropna())
                
#                 # 使用 GPT-4 进行总结
#                 summary = summarize_with_gpt4(all_text)
                
#                 # 将结果添加到列表中
#                 summaries.append({
#                     'Height_Category': height_category,
#                     'text': all_text,
#                     'summary': summary
#                 })
            
#             # 创建一个新的DataFrame来存储结果
#             result_df = pd.DataFrame(summaries)
            
#             # 保存结果到Excel文件，包含 filter_suffix 在文件名中
#             output_file = f"D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_{x}_{y}_{filter_suffix}_文本分类总结.xlsx"
#             result_df.to_excel(output_file, index=False)
#             print(f"Results saved to: {output_file}")

# print("All processing completed.")

# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(formatted_time)
# print(formatted_time)
# print(formatted_time)




# 以下使用了新的prompt

In [ ]:
# # 这里是用新版prompt,通过list读取,针对整体

# '''
# 这里是通过读取list形式, 来简化输入的
# '''

# import pandas as pd
# import os
# from tqdm import tqdm
# from openai import OpenAI



# # # 定义 x_list 和 y_list（只需要一次）
# # x_list = ['9734', '9733', '35434','21444', '1350','1349','1348','12005']  # 
# # x_list = ['9736', '9735', '12004']  # 男士春夏下
# # x_list = ['1348', '1349', '1350', '9733', '9734', '12005', '21444', '35434']  # 男士春夏上装
# # x_list = ['1354','1355','1356','9713','11988','12000','22355','28341','30786']  # 女士春夏上装
# # x_list = ['9715', '9716', '9717', '11991']  # 女士春夏下装
# # x_list = ['6914', '6916', '6917', '6918', '9775', '9776', '9777']  # 女鞋
# # x_list = ['1','2','3']

# # y_list = ['price']    # 添加所有需要的 Style 值

# # z = '男鞋_新分类_from_0501'



# # 设置API密钥和基础URL
# os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
# os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
# )

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True


# # 定义一个函数，使用GPT-4模型对文本进行自动摘要
# def summarize_with_gpt4(text):
#     response = client.chat.completions.create(
#         model="gpt-4o",  # 使用正确的模型名称，如果需要请更改
#         messages=[
#             {"role": "system", "content": f"你是一个非常优秀的电商分析师,现在有这样一个数据集data:\n\n{text}"},
#             {"role": "user", "content": """
#                 # Role 
#                 角色: 电商数据分析师。
#                 # Profile 
#                 简介: 我是一名电商数据分析师，我的工作是通过收集和整理各种信息，找出影响商品销量的因素，并形成报告。
#                 ## Background 
#                 背景: 现在有一些关于同一类商品的描述信息，我需要从中进行归纳总结，并基于一些前置的定义，找出这些描述信息都是从哪些维度切入的。
#                 ## Goals 
#                 目标: 基于我给到的商品描述信息数据集和前置的维度定义，归纳总结出描述的方向维度，需要特别关注与细化商品本身的卖点特性，并统计这些维度出现的频率。
#                 ## Definitions
#                 定义：
#                 1. 直接展示价格：直接展示价格信息，到手价，预估到手价，会员价等，通常包含上述前缀，¥+具体的价格数字或者具体的价格数字+元。
#                 2. 折扣信息：描述商品的折扣，通常包含具体的折扣数字+折。
#                 3. 直降信息：描述商品相较原价进行了大幅降价，通常包含直降、立减。
#                 4. 满减信息：描述若购买到一定金额，可以在此基础上进行金额优惠，通常包含满+具体的金额+减+具体的金额
#                 5. 赠品信息：描述若购买商品则会赠送服务或商品，通常包含赠、送
#                 6. 限时：描述商品促销的时间，通常包含活动时间段、活动开始时间、活动结束时间
#                 7. 品牌名称：描述商品的品牌名称
#                 8. 代言人信息：描述商品的代言人信息
#                 9. 价保：价格保护，通常包含价保
#                 10. 店铺背书：描述店铺的信息，通常包含旗舰店、自营
#                 11. 物流服务：描述商品所包含的物流服务，通常包含物流时效、运费险、物流名称、仓库名称、包邮
#                 12. 直接展示价格属于价格信息一级维度，折扣信息、直降信息、满减信息、赠品信息、限时属于价促活动一级维度，品牌名称、代言人信息属于品牌信息一级维度，价保、店铺背书、物流服务属于服务保障一级维度
#                 ## Constrains 
#                 约束条件: 1、时刻保持自己是电商数据分析师的角色，2、可以进行适当的联想和猜测，3、举例的时候禁止出现""，4、统计频率的时候请仔细仔细再仔细，5、若识别到的内容不在上述定义的维度中，可自行命名并统计，请不要忽视未被定义的维度，特别是关于商品本身的卖点信息描述
#                 ## Tone 
#                 语气风格: 正式的，客观的，科学的。
#                 ## Skills 
#                 技能: 1、你有出色的文本理解能力，能够理解输入数据的含义 2、你有出色的归纳总结能力，能够归纳总结出数据的描述维度 3、你也有出色的数据统计能力，能够精确的统计出各个维度出现的频次。
#                 ## OutputFormat 
#                 输出格式:以文字方式输出，一级维度，一级维度下具体内容和举例和频次，输出顺序按照价格信息、价促活动、品牌信息、服务保障、商品卖点进行输出，商品卖点为未定义维度，请你依照自己的知识库信息进行汇总输出，需要特别注意，是关于商品本身的描述，输出格式为1.价格信息 总频次 直接展示价格 频次 举例 以此类推,注意输出要精简，减少不必要的换行
#                     """}
#         ],
#     )
#     return response.choices[0].message.content.strip()


# # 遍历 x 和 y 的所有组合
# for x in x_list:
#     for y in y_list:
#         print(f"Processing: {x} - {y}")
        
#         # 读取Excel文件
#         file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification-{x}_{y}.xlsx'
#         if not os.path.exists(file_path):
#             print(f"File not found: {file_path}")
#             continue
        
#         df = pd.read_excel(file_path)
#         # df = df.dropna(subset=['structure'])

#         # 确保 'text' 列中的所有值都是字符串
#         df['text'] = df['text'].astype(str)
        
#         df = df[df.apply(filter_by_rectangle, axis=1)]
#         df_grouped = df.groupby(['Height_Category'])
        
#         # 遍历每个分组，合并文本并进行总结
#         summaries = []
        
#         # 遍历每个分组
#         for (height_category), group in tqdm(df_grouped):
#             # 合并该组的所有文本
#             all_text = " ".join(group['text'].dropna())
#             # print(f"Structure: {structure}")
#             # print(f"Height Category: {height_category}")
#             # print(f"Text: {all_text[:100]}...")  # 只打印前100个字符
            
#             # 使用 GPT-4 进行总结
#             try:
#                 summary = summarize_with_gpt4(all_text)
#                 # print(f"Summary: {summary[:100]}...")  # 只打印前100个字符
#             except Exception as e:
#                 # print(f"Error in summarization: {str(e)}")
#                 summary = "Error in summarization"
            
#             # 将结果添加到列表中
#             summaries.append({
#                 # 'structure': structure,
#                 'Height_Category': height_category,
#                 'text': all_text,
#                 'summary': summary
#             })
        
#         # 创建一个新的DataFrame来存储结果
#         result_df = pd.DataFrame(summaries)
        
#         # 保存结果到Excel文件
#         output_file = f"D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_{x}_{y}_文本分类总结-new.xlsx"
#         result_df.to_excel(output_file, index=False)
#         print(f"Results saved to: {output_file}")

# print("All processing completed.")


# import datetime
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(formatted_time)
# print(formatted_time)
# print(formatted_time)




In [ ]:
# # 这里是用新版prompt,通过list读取,针对的是品牌维度的分类


# '''
# 这里是通过读取list形式, 来简化输入的
# 添加了针对品牌维度的分类
# '''

# import pandas as pd
# import os
# from tqdm import tqdm
# from openai import OpenAI
# import datetime



# # 定义 filter_suffix_list
# # filter_suffix_list = ['filter_1.0','filter_2.0','filter_3.0','filter_4.0','filter_5.0','filter_6.0']  # 女鞋
# # filter_suffix_list = ['filter_1.0_2.0','filter_3.0_4.0','filter_5.0_6.0']  # 男鞋
# # filter_suffix_list = ['filter_1.0_2.0_3.0','filter_4.0_5.0_6.0']  # 男士&女士春夏上/下装



# # 设置API密钥和基础URL
# os.environ["OPENAI_API_KEY"] = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
# os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
# )

# def create_empty_file(file_path):
#     os.makedirs(os.path.dirname(file_path), exist_ok=True)
#     pd.DataFrame().to_excel(file_path, index=False)
#     print(f"Created empty file: {file_path}")

# def filter_by_rectangle(row):
#     right, bottom = 616 * 0.3, 616 * 0.2
#     if row['txt_x2'] < right and row['txt_y2'] < bottom:
#         return False
#     return True

# # 定义一个函数，使用GPT-4模型对文本进行自动摘要
# def summarize_with_gpt4(text):
#     try:
#         response = client.chat.completions.create(
#             model="gpt-4o",  # 使用正确的模型名称，如果需要请更改
#             messages=[
#                 {"role": "system", "content": f"你是一个非常优秀的电商分析师,现在有这样一个数据集data:\n\n{text}"},
#                 {"role": "user", "content": """
#                     # Role 
#                     角色: 电商数据分析师。
#                     # Profile 
#                     简介: 我是一名电商数据分析师，我的工作是通过收集和整理各种信息，找出影响商品销量的因素，并形成报告。
#                     ## Background 
#                     背景: 现在有一些关于同一类商品的描述信息，我需要从中进行归纳总结，并基于一些前置的定义，找出这些描述信息都是从哪些维度切入的。
#                     ## Goals 
#                     目标: 基于我给到的商品描述信息数据集和前置的维度定义，归纳总结出描述的方向维度，需要特别关注与细化商品本身的卖点特性，并统计这些维度出现的频率。
#                     ## Definitions
#                     定义：
#                     1. 直接展示价格：直接展示价格信息，到手价，预估到手价，会员价等，通常包含上述前缀，¥+具体的价格数字或者具体的价格数字+元。
#                     2. 折扣信息：描述商品的折扣，通常包含具体的折扣数字+折。
#                     3. 直降信息：描述商品相较原价进行了大幅降价，通常包含直降、立减。
#                     4. 满减信息：描述若购买到一定金额，可以在此基础上进行金额优惠，通常包含满+具体的金额+减+具体的金额
#                     5. 赠品信息：描述若购买商品则会赠送服务或商品，通常包含赠、送
#                     6. 限时：描述商品促销的时间，通常包含活动时间段、活动开始时间、活动结束时间
#                     7. 品牌名称：描述商品的品牌名称
#                     8. 代言人信息：描述商品的代言人信息
#                     9. 价保：价格保护，通常包含价保
#                     10. 店铺背书：描述店铺的信息，通常包含旗舰店、自营
#                     11. 物流服务：描述商品所包含的物流服务，通常包含物流时效、运费险、物流名称、仓库名称、包邮
#                     12. 直接展示价格属于价格信息一级维度，折扣信息、直降信息、满减信息、赠品信息、限时属于价促活动一级维度，品牌名称、代言人信息属于品牌信息一级维度，价保、店铺背书、物流服务属于服务保障一级维度
#                     ## Constrains 
#                     约束条件: 1、时刻保持自己是电商数据分析师的角色，2、可以进行适当的联想和猜测，3、举例的时候禁止出现""，4、统计频率的时候请仔细仔细再仔细，5、若识别到的内容不在上述定义的维度中，可自行命名并统计，请不要忽视未被定义的维度，特别是关于商品本身的卖点信息描述
#                     ## Tone 
#                     语气风格: 正式的，客观的，科学的。
#                     ## Skills 
#                     技能: 1、你有出色的文本理解能力，能够理解输入数据的含义 2、你有出色的归纳总结能力，能够归纳总结出数据的描述维度 3、你也有出色的数据统计能力，能够精确的统计出各个维度出现的频次。
#                     ## OutputFormat 
#                     输出格式:以文字方式输出，一级维度，一级维度下具体内容和举例和频次，输出顺序按照价格信息、价促活动、品牌信息、服务保障、商品卖点进行输出，商品卖点为未定义维度，请你依照自己的知识库信息进行汇总输出，需要特别注意，是关于商品本身的描述，输出格式为1.价格信息 总频次 直接展示价格 频次 举例 以此类推,注意输出要精简，减少不必要的换行
#                         """}
#             ],
#         )
#         return response.choices[0].message.content.strip()
#     except Exception as e:
#         print(f"Error in summarization: {str(e)}")
#         return ""

# # 遍历 x, y 和 filter_suffix 的所有组合
# for x in x_list:
#     for y in y_list:
#         for filter_suffix in filter_suffix_list:
#             print(f"Processing: {x} - {y} - {filter_suffix}")
            
#             # 读取Excel文件
#             file_path = f'D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_word_classification-{x}_{y}_{filter_suffix}.xlsx'
#             if not os.path.exists(file_path):
#                 print(f"File not found: {file_path}")
#                 create_empty_file(file_path)
#                 continue
            
#             try:
#                 df = pd.read_excel(file_path)
#             except Exception as e:
#                 print(f"Error reading file {file_path}: {str(e)}")
#                 create_empty_file(file_path)
#                 continue

#             if df.empty:
#                 print(f"Empty DataFrame for {file_path}")
#                 continue

#             # 确保 'text' 列中的所有值都是字符串
#             df['text'] = df['text'].astype(str)
            
#             # 根据 filter_suffix 筛选数据
#             filter_values = [float(val) for val in filter_suffix.split('_')[1:]]
#             df = df[df['最终分层'].isin(filter_values)]
            
#             df = df[df.apply(filter_by_rectangle, axis=1)]
#             df_grouped = df.groupby(['Height_Category'])
            
#             # 遍历每个分组，合并文本并进行总结
#             summaries = []
            
#             # 遍历每个分组
#             for (height_category), group in tqdm(df_grouped):
#                 # 合并该组的所有文本
#                 all_text = " ".join(group['text'].dropna())
                
#                 # 使用 GPT-4 进行总结
#                 summary = summarize_with_gpt4(all_text)
                
#                 # 将结果添加到列表中
#                 summaries.append({
#                     'Height_Category': height_category,
#                     'text': all_text,
#                     'summary': summary
#                 })
            
#             # 创建一个新的DataFrame来存储结果
#             result_df = pd.DataFrame(summaries)
            
#             # 保存结果到Excel文件，包含 filter_suffix 在文件名中
#             output_file = f"D://code//data//{path}//{z}//{x}//grounding_output//{y}//50%_{x}_{y}_{filter_suffix}_文本分类总结-new.xlsx"
#             result_df.to_excel(output_file, index=False)
#             print(f"Results saved to: {output_file}")

# print("All processing completed.")

# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
# print(formatted_time)
# print(formatted_time)
# print(formatted_time)


